# 2) Notebook: configuración y conexión

In [1]:
# --- Configuración base ---
import os
from datetime import datetime, timedelta
import random
import numpy as np
import pandas as pd
from faker import Faker
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [2]:
# Oracle / SQLAlchemy
import oracledb  # driver nativo
from sqlalchemy import (
    create_engine,
    String,
    Integer,
    Numeric,
    Date,
    DateTime,
    CheckConstraint,
    ForeignKey,
    UniqueConstraint,
    Sequence,
)
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session

In [3]:
# Validación
import pandera as pa
from pandera import Column, Check

fake = Faker("es_ES")

In [4]:
# --- Credenciales Oracle (ajusta a tu entorno) ---
USER = "TMPRD"
PASS = "TMPRD"
HOST = "localhost"
PORT = 1521
SERVICE = "orclaguai"  # o tu servicio (ORCL, etc.)

# Cadena SQLAlchemy con oracledb
engine = create_engine(
    f"oracle+oracledb://{USER}:{PASS}@{HOST}:{PORT}/?service_name={SERVICE}",
    max_identifier_length=128,  # por seguridad con nombres largos
    pool_pre_ping=True,
)

In [5]:
from sqlalchemy.exc import SQLAlchemyError

try:
    with engine.connect() as conn:
        print("Conexión exitosa a Oracle.")
except SQLAlchemyError as e:
    print("Error al conectar a Oracle:", e)

Conexión exitosa a Oracle.


# 3) DDL – Tablas (ORM SQLAlchemy)

Nota: he simplificado tipos/longitudes. Ajusta a tus estándares.

Incluye PK/FK/Checks mínimos para ventas y traslados (las que usaste en tus SQL).

In [6]:
class Base(DeclarativeBase):
    pass

In [7]:
# ====== MAESTROS ======
class SZG300(Base):  # Tipo Cliente
    __tablename__ = "SZG300"
    ZG_CODIGO: Mapped[str] = mapped_column(String(4), primary_key=True)
    ZG_DESC: Mapped[str] = mapped_column(String(60), nullable=False)
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")


class SA1300(Base):
    __tablename__ = "SA1300"
    A1_COD: Mapped[str] = mapped_column(String(10), primary_key=True)
    A1_NOME: Mapped[str] = mapped_column(String(120), nullable=False)
    A1_CGC: Mapped[str] = mapped_column(String(20), nullable=True)
    A1_UTPCLI: Mapped[str] = mapped_column(
        String(4), ForeignKey("SZG300.ZG_CODIGO"), nullable=False
    )
    A1_LOJA: Mapped[str] = mapped_column(
        String(4), nullable=False
    )  # Esta columna no debe ser parte de la PK
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")


class SB1300(Base):  # Productos
    __tablename__ = "SB1300"
    B1_COD: Mapped[str] = mapped_column(String(15), primary_key=True)
    B1_DESC: Mapped[str] = mapped_column(String(120), nullable=False)
    B1_UM: Mapped[str] = mapped_column(String(6), nullable=False)
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")


class SZH300(Base):  # Almacenes/Móvil
    __tablename__ = "SZH300"
    ZH_ALM: Mapped[str] = mapped_column(String(6), primary_key=True)
    ZH_DESC: Mapped[str] = mapped_column(String(80), nullable=False)


class CTT300(Base):  # Segmentos / Centros de Costo
    __tablename__ = "CTT300"
    CTT_CUSTO: Mapped[str] = mapped_column(String(12), primary_key=True)
    CTT_DESC01: Mapped[str] = mapped_column(String(120), nullable=False)

In [8]:
# ====== PEDIDOS ======
sc5300_id_seq = Sequence("SC5300_SEQ", start=100, increment=1)
class SC5300(Base):  # Cabecera pedido
    __tablename__ = "SC5300"
    C5_NUM: Mapped[str] = mapped_column(String(10), primary_key=True) # MODIFIED = MODIFIQUE
    C5_CLIENTE: Mapped[str] = mapped_column(String(10), nullable=False)
    C5_CLIENTE: Mapped[str] = mapped_column(
        String(10), ForeignKey("SA1300.A1_COD"), nullable=False
    )
    C5_EMISSAO: Mapped[datetime] = mapped_column(Date, nullable=False)
    C5_XTIPO: Mapped[str] = mapped_column(String(1), nullable=False)  # 1..8
    C5_MOEDA: Mapped[str] = mapped_column(String(3), nullable=False)
    C5_LOJACLI: Mapped[str] = mapped_column(String(4), nullable=True)
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")

    __table_args__ = (
          CheckConstraint(
            "C5_XTIPO in ('1','2','3','4','5','6','7','8')", name="CK_SC5_XTIPO"
        ),
    )

sc6300_id_seq = Sequence("SC6300_SEQ", start=100, increment=1)
class SC6300(Base):  # Detalle pedido
    __tablename__ = "SC6300"
    C6_ID: Mapped[int] = mapped_column(Integer, sc6300_id_seq, primary_key=True)
    C6_NUM: Mapped[str] = mapped_column(
        String(10), ForeignKey("SC5300.C5_NUM"), nullable=False
    )
    C6_ITEM: Mapped[int] = mapped_column(Integer, nullable=False)
    C6_PRODUTO: Mapped[str] = mapped_column(
        String(15), ForeignKey("SB1300.B1_COD"), nullable=False
    )
    C6_DESCRI: Mapped[str] = mapped_column(String(120), nullable=True)
    C6_UM: Mapped[str] = mapped_column(String(6), nullable=False)
    C6_QTDVEN: Mapped[float] = mapped_column(Numeric(15, 3), nullable=False)
    C6_QTDENT: Mapped[float] = mapped_column(Numeric(15, 3), nullable=True, default=0)
    C6_PRCVEN: Mapped[float] = mapped_column(Numeric(15, 4), nullable=False)
    C6_VALOR: Mapped[float] = mapped_column(Numeric(15, 2), nullable=False)
    C6_TES: Mapped[str] = mapped_column(String(5), nullable=True)
    C6_LOCAL: Mapped[str] = mapped_column(String(6), ForeignKey("SZH300.ZH_ALM"))
    C6_LOCDEST: Mapped[str] = mapped_column(String(6), ForeignKey("SZH300.ZH_ALM"))
    C6_CLI: Mapped[str] = mapped_column(String(10), nullable=False)
    C6_LOJA: Mapped[str] = mapped_column(String(4), nullable=False)
    C6_FILIAL: Mapped[str] = mapped_column(String(4), nullable=True)
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")

    __table_args__ = (UniqueConstraint("C6_NUM", "C6_ITEM", name="UQ_SC6_NUM_ITEM"),)

In [9]:
# ====== APROBACIÓN (SC9 simplificado) ======
sc9300_id_seq = Sequence("SC900_SEQ", start=100, increment=1)
class SC900(Base):
    __tablename__ = "SC900"
    C9_ID: Mapped[int] = mapped_column(Integer, sc9300_id_seq, primary_key=True)
    C9_NUM: Mapped[str] = mapped_column(String(10), nullable=False)  # pedido
    C9_ITEM: Mapped[int] = mapped_column(Integer, nullable=False)  # ítem
    C9_STATUS: Mapped[str] = mapped_column(
        String(1), nullable=False
    )  # A=aprobado, B=bloq, P=pend
    C9_QUANT: Mapped[float] = mapped_column(Numeric(15, 3), nullable=False)
    C9_PRECO: Mapped[float] = mapped_column(Numeric(15, 4), nullable=False)
    C9_USR: Mapped[str] = mapped_column(String(30), nullable=False)
    C9_DATA: Mapped[datetime] = mapped_column(Date, nullable=False)

    __table_args__ = (
        CheckConstraint("C9_STATUS in ('A','B','P')", name="CK_SC9_STATUS"),
        UniqueConstraint("C9_NUM", "C9_ITEM", name="UQ_SC9_NUM_ITEM"),
    )

In [10]:
# ====== REMITOS / FACTURAS (detalle + cabecera) ======
sd2300_id_seq = Sequence("SD2300_SEQ", start=100, increment=1)
class SD2300(Base):
    __tablename__ = "SD2300"
    D2_ID: Mapped[int] = mapped_column(Integer, sd2300_id_seq, primary_key=True)
    D2_TIPODOC: Mapped[str] = mapped_column(
        String(2), nullable=False
    )  # '50'=Remito, '01'=Factura (detalle)
    D2_DOC: Mapped[str] = mapped_column(String(12), nullable=False)
    D2_SERIE: Mapped[str] = mapped_column(String(6), nullable=False)
    D2_FILIAL: Mapped[str] = mapped_column(String(4), nullable=True)

    D2_CLIENTE: Mapped[str] = mapped_column(String(10), nullable=True)
    D2_LOJA: Mapped[str] = mapped_column(String(4), nullable=True)

    D2_COD: Mapped[str] = mapped_column(
        String(15), ForeignKey("SB1300.B1_COD"), nullable=False
    )
    D2_UM: Mapped[str] = mapped_column(String(6), nullable=False)
    D2_LOCAL: Mapped[str] = mapped_column(String(6), ForeignKey("SZH300.ZH_ALM"))
    D2_QUANT: Mapped[float] = mapped_column(Numeric(15, 3), nullable=False)
    D2_QTSEGUM: Mapped[float] = mapped_column(Numeric(15, 3), nullable=True)
    D2_PRCVEN: Mapped[float] = mapped_column(Numeric(15, 4), nullable=True)
    D2_TOTAL: Mapped[float] = mapped_column(Numeric(15, 2), nullable=True)
    D2_CUSTO1: Mapped[float] = mapped_column(Numeric(15, 2), nullable=True)
    D2_TES: Mapped[str] = mapped_column(String(5), nullable=True)
    D2_VALIMP5: Mapped[float] = mapped_column(Numeric(15, 2), nullable=True)
    D2_CCUSTO: Mapped[str] = mapped_column(
        String(12), ForeignKey("CTT300.CTT_CUSTO"), nullable=True
    )
    D2_EMISSAO: Mapped[datetime] = mapped_column(Date, nullable=False)
    D2_DTDIGIT: Mapped[datetime] = mapped_column(Date, nullable=False)

    # Enlaces a pedido
    D2_PEDIDO: Mapped[str] = mapped_column(String(10), nullable=True)
    D2_SEQUEN: Mapped[int] = mapped_column(Integer, nullable=True)
    D2_ITEMPV: Mapped[int] = mapped_column(Integer, nullable=True)

    # Enlaces remito<->factura
    D2_REMITO: Mapped[str] = mapped_column(String(12), nullable=True)
    D2_SERIREM: Mapped[str] = mapped_column(String(6), nullable=True)
    D2_ITEMREM: Mapped[int] = mapped_column(Integer, nullable=True)

    __table_args__ = (
        CheckConstraint("D2_TIPODOC in ('01','50')", name="CK_SD2_TIPODOC"),
    )

sc2300_id_seq = Sequence("SF2300_SEQ", start=100, increment=1)
class SF2300(Base):  # Cabecera Factura
    __tablename__ = "SF2300"
    F2_ID: Mapped[int] = mapped_column(Integer, sc2300_id_seq, primary_key=True)
    F2_DOC: Mapped[str] = mapped_column(String(12), nullable=False)
    F2_SERIE: Mapped[str] = mapped_column(String(6), nullable=False)
    F2_DTDIGIT: Mapped[datetime] = mapped_column(Date, nullable=False)
    F2_CLIENTE: Mapped[str] = mapped_column(String(10), nullable=False)
    F2_LOJA: Mapped[str] = mapped_column(String(4), nullable=False)
    F2_MOEDA: Mapped[str] = mapped_column(String(3), nullable=False)
    F2_NUMAUT: Mapped[str] = mapped_column(String(30), nullable=True)
    F2_CODCTR: Mapped[str] = mapped_column(String(30), nullable=True)
    F2_UNIT: Mapped[str] = mapped_column(String(20), nullable=True)  # CI/NIT alterno
    F2_UNOME: Mapped[str] = mapped_column(String(120), nullable=True)  # Nombre alterno

    __table_args__ = (UniqueConstraint("F2_DOC", "F2_SERIE", name="UQ_SF2_DOC_SERIE"),)

In [11]:
# ====== BALANZA / TRANSPORTE ======
pb7300_id_seq = Sequence("PB7300_SEQ", start=100, increment=1)
class PB7300(Base):
    __tablename__ = "PB7300"
    PB7_ID: Mapped[int] = mapped_column(Integer, pb7300_id_seq, primary_key=True)
    PB7_FILIAL: Mapped[str] = mapped_column(String(4), nullable=False)
    PB7_DATA: Mapped[datetime] = mapped_column(Date, nullable=False)
    PB7_DSAIDA: Mapped[datetime] = mapped_column(Date, nullable=True)

    PB7_PEDIDO: Mapped[str] = mapped_column(String(10), nullable=True)
    PB7_SEQUEN: Mapped[int] = mapped_column(Integer, nullable=True) 
    PB7_ITEMPV: Mapped[int] = mapped_column(Integer, nullable=True)
    PB7_PRODUT: Mapped[str] = mapped_column(
        String(15), ForeignKey("SB1300.B1_COD"), nullable=False
    )
    PB7_QTDE: Mapped[float] = mapped_column(Numeric(15, 3), nullable=False)

    PB7_STATUS: Mapped[str] = mapped_column(
        String(1), nullable=False
    )  # E/S/F (Entrada/Salida/Anulado)
    PB7_PBRUTO: Mapped[float] = mapped_column(Numeric(15, 3), nullable=True)
    PB7_PTARA: Mapped[float] = mapped_column(Numeric(15, 3), nullable=True)

    PB7_PLACA: Mapped[str] = mapped_column(String(12), nullable=True)
    PB7_CHOFER: Mapped[str] = mapped_column(String(80), nullable=True)
    PB7_TELEFO: Mapped[str] = mapped_column(String(20), nullable=True)
    PB7_DOCUME: Mapped[str] = mapped_column(String(20), nullable=True)

    PB7_XDESP: Mapped[str] = mapped_column(String(20), nullable=True)
    PB7_LOCENT: Mapped[str] = mapped_column(String(10), nullable=True)
    PB7_XZAFRA: Mapped[str] = mapped_column(String(10), nullable=True)
    PB7_XOBS: Mapped[str] = mapped_column(String(100), nullable=True)
    PB7_OBSERV: Mapped[str] = mapped_column(String(100), nullable=True)

    __table_args__ = (
        CheckConstraint("PB7_STATUS in ('E','S','F')", name="CK_PB7_STATUS"),
    )

In [12]:
# ====== MOVIMIENTOS DE STOCK / PRODUCCIÓN / TRASLADOS ======
sd3300_id_seq = Sequence("SD3300_SEQ", start=100, increment=1)
class SD3300(Base):
    __tablename__ = "SD3300"
    D3_ID: Mapped[int] = mapped_column(Integer, sd3300_id_seq, primary_key=True)
    D3_FILIAL: Mapped[str] = mapped_column(String(4), nullable=True)
    D3_DOC: Mapped[str] = mapped_column(String(12), nullable=True)
    D3_XNROTRA: Mapped[str] = mapped_column(String(14), nullable=True)
    D3_COD: Mapped[str] = mapped_column(
        String(15), ForeignKey("SB1300.B1_COD"), nullable=False
    )
    D3_UM: Mapped[str] = mapped_column(String(6), nullable=False)
    D3_QUANT: Mapped[float] = mapped_column(Numeric(15, 3), nullable=False)
    D3_QTSEGUM: Mapped[float] = mapped_column(Numeric(15, 3), nullable=True)
    D3_CUSTO1: Mapped[float] = mapped_column(Numeric(15, 4), nullable=True)
    D3_LOCAL: Mapped[str] = mapped_column(String(6), ForeignKey("SZH300.ZH_ALM"))
    D3_EMISSAO: Mapped[datetime] = mapped_column(Date, nullable=False)
    D3_TM: Mapped[str] = mapped_column(String(3), nullable=False)  # '001','008','501'
    D3_XORIGEN: Mapped[str] = mapped_column(String(10), nullable=True)
    D3_USUARIO: Mapped[str] = mapped_column(String(30), nullable=True)
    D3_ESTORNO: Mapped[str] = mapped_column(String(1), nullable=True, default="N")  # S/N
    D_E_L_E_T_: Mapped[str] = mapped_column(String(1), nullable=False, default=" ")

    __table_args__ = (
        CheckConstraint("D3_TM in ('001','008','501')", name="CK_SD3_TM"),
    )

In [13]:
Base.metadata.drop_all(engine)
print("Tablas eliminadas.")

Tablas eliminadas.


In [14]:
# Crear todas las tablas
Base.metadata.create_all(engine)
print("Tablas creadas.")

Tablas creadas.


# 4) Reglas de negocio (validaciones con Pandera)

Principales reglas que validaremos:

- SC5/SC6: pedido debe tener cliente existente; tipo pedido válido; montos coherentes.

- SC9: ítem pedido aprobado antes de remito/factura.

- SD3:

    - TM='001' = ingreso; TM='501' = salida origen traslado; TM='008' = entrada destino traslado.

    - Traslado siempre tiene par (501 y 008) con mismo XNROTRA, COD, QUANT, EMISSAO.

- SD2 (remito): si hay entrega (PB7), cantidades consistentes; y remito solo para ítems aprobados.

- SF2 (factura): solo si hay remito(s) asociados (o regla que definas).

In [15]:
# --- Pandera schemas (ejemplo resumido) ---
SC5_schema = pa.DataFrameSchema(
    {
        "C5_NUM": Column(str, Check.str_length(1, 10)),
        "C5_CLIENTE": Column(str),
        "C5_LOJACLI": Column(str),
        "C5_EMISSAO": Column("datetime64[ns]", coerce=True),
        "C5_XTIPO": Column(str, Check.isin(list("12345678"))),
        "C5_MOEDA": Column(str, Check.isin(["1", "2", "USD", "BOB"])),
    }
)

SC6_schema = pa.DataFrameSchema(
    {
        "C6_NUM": Column(str),
        "C6_ITEM": Column(int, Check.ge(1)),
        "C6_PRODUTO": Column(str),
        "C6_UM": Column(str),
        "C6_QTDVEN": Column(float, Check.gt(0)),
        "C6_PRCVEN": Column(float, Check.ge(0)),
        "C6_VALOR": Column(float, Check.ge(0)),
        "C6_LOCAL": Column(str, nullable=True),
        "C6_LOCDEST": Column(str, nullable=True),
        "C6_CLI": Column(str),
        "C6_LOJA": Column(str),
    }
)

SD3_schema = pa.DataFrameSchema(
    {
        "D3_COD": Column(str),
        "D3_UM": Column(str),
        "D3_QUANT": Column(float, Check.gt(0)),
        "D3_EMISSAO": Column("datetime64[ns]", coerce=True),
        "D3_TM": Column(str, Check.isin(["001", "501", "008"])),
        "D3_LOCAL": Column(str),
    }
)

c:\Users\jelo2\Documents\GitHub\dbcopilot\faker\.venv\Lib\site-packages\pandera\_pandas_deprecated.py:149: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# 5) Generación de datos fake (orden de carga)

## 5.1 Maestros

In [16]:
with Session(engine) as s, s.begin():
    # Tipos de cliente
    tipos = [
        ("01", "MERCADO"),
        ("02", "MAYORISTA"),
        ("03", "PROVINCIA"),
        ("04", "INDUSTRIA"),
        ("06", "INTERIOR"),
        ("08", "EXPORTACION"),
        ("20", "INSTITUCIONES"),
        ("26", "INTERESES"),
    ]
    for c, d in tipos:
        s.merge(SZG300(ZG_CODIGO=c, ZG_DESC=d))

    # Segmentos
    segs = [
        ("100", "Mercado"),
        ("200", "Mayorista"),
        ("300", "Exportación"),
        ("400", "Industrial"),
    ]
    for c, d in segs:
        s.merge(CTT300(CTT_CUSTO=c, CTT_DESC01=d))

    # Almacenes
    alms = [
        ("20", "Central"),
        ("29", "Mayorista"),
        ("55", "Super"),
        ("63", "Mercado"),
        ("74", "Minorista"),
        ("90", "Alcohol"),
        ("91", "Bagazo"),
    ]
    for c, d in alms:
        s.merge(SZH300(ZH_ALM=c, ZH_DESC=d))

    # Productos (PA01 = alcohol, PA02 = azúcar, PA03 = bagazo)
    prods = [
        ("PA010001", "ALCOHOL HIDRATADO", "LT"),
        ("PA020121", "AZUCAR 1KG", "BL"),
        ("PA020211", "AZUCAR 50KG", "BL"),
        ("PA030001", "BAGAZO", "KG"),
    ]
    for cod, desc, um in prods:
        s.merge(SB1300(B1_COD=cod, B1_DESC=desc, B1_UM=um))

    # Clientes
    for _ in range(20):
        cod = f"{random.randint(1,999999):06d}"
        loja = f"{random.randint(1,2):02d}"
        tipo = random.choice(tipos)[0]
        s.merge(
            SA1300(
                A1_COD=cod,
                A1_LOJA=loja,
                A1_NOME=fake.company(),
                A1_CGC=str(random.randint(1000000, 99999999)),
                A1_UTPCLI=tipo,
            )
        )
print("Maestros cargados.")

Maestros cargados.


## 5.2 Producción / Traslados (SD3)

In [17]:
start = datetime(2020, 3, 1).date()
dates = [start + timedelta(days=i) for i in range(10)]

rows = []
# Ingresos de producción AZUCAR/ALCOHOL (TM=001)
for d in dates:
    qty1 = random.uniform(100.0, 500.0)
    rows.append(
        dict(
            D3_COD="PA020211",
            D3_UM="BL",
            D3_QUANT=qty1,  # Use uniform for float values
            D3_QTSEGUM=qty1 + round(random.uniform(0.01, 0.99), 2),
            D3_LOCAL="20",
            D3_EMISSAO=d,
            D3_TM="001",
            D3_XNROTRA=None,  # ¡CAMBIO AQUI!
        )
    )
    qty2 = random.randint(50000, 120000)
    rows.append(
        dict(
            D3_COD="PA010001",
            D3_UM="LT",
            D3_QUANT=qty2,
            D3_QTSEGUM=qty2 + round(random.uniform(0.01, 0.99), 2),
            D3_LOCAL="90",
            D3_EMISSAO=d,
            D3_TM="001",
            D3_XNROTRA=None,  # ¡CAMBIO AQUI!
        )
    )

# Traslados internos (501/008) – mover a otros almacenes
for d in dates[::2]:
    xnrotra = f"TR{d.strftime('%Y%m%d')}{random.randint(100,999)}"
    qty = random.uniform(50.0, 150.0)
    for cod, um, origen, destino in [
        ("PA020211", "BL", "20", "29"),
        ("PA010001", "LT", "90", "90"),
    ]:
        rows.append(
            dict(
                D3_COD=cod,
                D3_UM=um,
                D3_QUANT=qty,
                D3_QTSEGUM=qty + round(random.uniform(0.01, 0.99), 2),
                D3_LOCAL=origen,
                D3_EMISSAO=d,
                D3_TM="501",
                D3_XNROTRA=xnrotra,
                # D3_ESTORNO="N",
            )
        )
        rows.append(
            dict(
                D3_COD=cod,
                D3_UM=um,
                D3_QUANT=qty,
                D3_QTSEGUM=qty + round(random.uniform(0.01, 0.99), 2),
                D3_LOCAL=destino,
                D3_EMISSAO=d,
                D3_TM="008",
                D3_XNROTRA=xnrotra,
                # D3_ESTORNO="N",
            )
        )

df_sd3 = pd.DataFrame(rows)
SD3_schema.validate(df_sd3)  # valida reglas básicas

with Session(engine) as s, s.begin():
    for r in df_sd3.to_dict(orient="records"):
        s.add(
            SD3300(**{k: r.get(k) for k in SD3300.__table__.columns.keys() if k in r})
        )
print("SD3 cargado.")

SD3 cargado.


## 5.3 Pedidos (SC5/SC6) → Aprobación (SC9)

In [18]:
# Elige clientes y productos válidos
with Session(engine) as s:
    clientes = s.query(SA1300).all()
    productos = s.query(SB1300).all()

pedidos = []
detalles = []
aprob = []

for n in range(20):
    cte = random.choice(clientes)
    num = f"{10500+n:06d}"
    c5 = dict(
        C5_NUM=num,
        C5_CLIENTE=cte.A1_COD,
        C5_LOJACLI=cte.A1_LOJA,
        C5_EMISSAO=start + timedelta(days=random.randint(0, 5)),
        C5_XTIPO=random.choice(list("158")),  # normal/local/export simplificado
        C5_MOEDA=random.choice(["1", "USD"]),
    )
    pedidos.append(c5)
    # 1..3 ítems
    for item in range(1, random.randint(2, 4)):
        prod = random.choice(productos)
        qty = (
            random.uniform(1.0, 50.0)
            if prod.B1_UM != "LT"
            else random.uniform(1000.0, 80000.0)
        )
        prc = round(random.uniform(0.5, 250.0), 2)
        val = round(qty * prc, 2)
        detalles.append(
            dict(
                C6_NUM=num,
                C6_ITEM=item,
                C6_PRODUTO=prod.B1_COD,
                C6_UM=prod.B1_UM,
                C6_QTDVEN=qty,
                C6_PRCVEN=prc,
                C6_VALOR=val,
                C6_LOCAL=random.choice(["20", "29", "55", "63", "74", "90", "91"]),
                C6_LOCDEST=random.choice(["20", "29", "55", "63", "74", "90", "91"]),
                C6_CLI=cte.A1_COD,
                C6_LOJA=cte.A1_LOJA,
                C6_FILIAL="0001",
            )
        )
        aprob.append(
            dict(
                C9_NUM=num,
                C9_ITEM=item,
                C9_STATUS=random.choice(["A", "A", "P"]),  # mayoría aprobados
                C9_QUANT=qty,
                C9_PRECO=prc,
                C9_USR="aprobador",
                C9_DATA=start,
            )
        )

df_sc5 = pd.DataFrame(pedidos)
df_sc6 = pd.DataFrame(detalles)
df_sc9 = pd.DataFrame(aprob)

SC5_schema.validate(df_sc5)
SC6_schema.validate(df_sc6)

with Session(engine) as s, s.begin():
    for r in df_sc5.to_dict(orient="records"):
        s.add(SC5300(**r))
    for r in df_sc6.to_dict(orient="records"):
        s.add(SC6300(**r))
    for r in df_sc9.to_dict(orient="records"):
        s.add(SC900(**r))
print("Pedidos y aprobaciones cargados.")

Pedidos y aprobaciones cargados.


## 5.4 Remitos (SD2) ↔ PB7 (Despacho) → Facturas (SF2 + SD2 01)

In [19]:
# Generar remitos solo para ítems Aprobados (SC9.STATUS='A')
with Session(engine) as s, s.begin():
    aprobados = s.query(SC900).filter(SC900.C9_STATUS == "A").all()
    for ap in aprobados:
        # Remito (50)
        d2 = SD2300(
            D2_TIPODOC="50",
            D2_DOC=f"R{random.randint(10000,99999)}",
            D2_SERIE="001",
            D2_FILIAL="0001",
            D2_CLIENTE=s.query(SC6300)
            .filter(SC6300.C6_NUM == ap.C9_NUM, SC6300.C6_ITEM == ap.C9_ITEM)
            .first()
            .C6_CLI,
            D2_LOJA=s.query(SC6300)
            .filter(SC6300.C6_NUM == ap.C9_NUM, SC6300.C6_ITEM == ap.C9_ITEM)
            .first()
            .C6_LOJA,
            D2_COD=s.query(SC6300)
            .filter(SC6300.C6_NUM == ap.C9_NUM, SC6300.C6_ITEM == ap.C9_ITEM)
            .first()
            .C6_PRODUTO,
            D2_UM=s.query(SB1300)
            .filter(
                SB1300.B1_COD
                == s.query(SC6300)
                .filter(SC6300.C6_NUM == ap.C9_NUM, SC6300.C6_ITEM == ap.C9_ITEM)
                .first()
                .C6_PRODUTO
            )
            .first()
            .B1_UM,
            D2_LOCAL=s.query(SC6300)
            .filter(SC6300.C6_NUM == ap.C9_NUM, SC6300.C6_ITEM == ap.C9_ITEM)
            .first()
            .C6_LOCAL,
            D2_QUANT=ap.C9_QUANT,
            D2_QTSEGUM=None,
            D2_PRCVEN=ap.C9_PRECO,
            D2_TOTAL=round(float(ap.C9_QUANT) * float(ap.C9_PRECO), 2),
            D2_CUSTO1=round(float(ap.C9_QUANT) * float(ap.C9_PRECO) * 0.7, 2),  # coste estimado
            D2_TES="501",
            D2_CCUSTO=random.choice(["100", "200", "300", "400"]),
            D2_EMISSAO=start,
            D2_DTDIGIT=start,
            D2_PEDIDO=ap.C9_NUM,
            D2_SEQUEN=ap.C9_ITEM,
            D2_ITEMPV=ap.C9_ITEM,
        )
        s.add(d2)
        s.flush()  # get D2_ID

        # PB7 (despacho real)
        s.add(
            PB7300(
                PB7_FILIAL="0001",
                PB7_DATA=start,
                PB7_DSAIDA=start,
                PB7_PEDIDO=ap.C9_NUM,
                PB7_SEQUEN=ap.C9_ITEM,
                PB7_ITEMPV=ap.C9_ITEM,
                PB7_PRODUT=d2.D2_COD,
                PB7_QTDE=d2.D2_QUANT,  # igual a remito simple
                PB7_STATUS="S",
                PB7_PBRUTO=float(d2.D2_QUANT) + random.uniform(0, 10),
                PB7_PTARA=random.uniform(0, 5),
                PB7_PLACA="ABC-123",
                PB7_CHOFER=fake.name(),
                PB7_TELEFO=fake.phone_number(),
                PB7_DOCUME=str(random.randint(1000000, 9999999)),
                PB7_XDESP=str(random.randint(10000, 99999)),
                PB7_LOCENT=d2.D2_LOCAL,
                PB7_XZAFRA="2020",
            )
        )

        # Factura (cabecera) + detalle (01)
        fdoc = f"F{random.randint(10000,99999)}"
        s.add(
            SF2300(
                F2_DOC=fdoc,
                F2_SERIE="001",
                F2_DTDIGIT=start,
                F2_CLIENTE=d2.D2_CLIENTE,
                F2_LOJA=d2.D2_LOJA,
                F2_MOEDA="1",
                F2_NUMAUT="AUTO123",
                F2_CODCTR="CTR001",
                F2_UNIT=None,
                F2_UNOME=None,
            )
        )
        s.add(
            SD2300(
                D2_TIPODOC="01",
                D2_DOC=fdoc,
                D2_SERIE="001",
                D2_FILIAL="0001",
                D2_CLIENTE=d2.D2_CLIENTE,
                D2_LOJA=d2.D2_LOJA,
                D2_COD=d2.D2_COD,
                D2_UM=d2.D2_UM,
                D2_LOCAL=d2.D2_LOCAL,
                D2_QUANT=d2.D2_QUANT,
                D2_QTSEGUM=None,
                D2_PRCVEN=d2.D2_PRCVEN,
                D2_TOTAL=d2.D2_TOTAL,
                D2_TES=d2.D2_TES,
                D2_CCUSTO=d2.D2_CCUSTO,
                D2_EMISSAO=start,
                D2_DTDIGIT=start,
                D2_REMITO=d2.D2_DOC,
                D2_SERIREM=d2.D2_SERIE,
                D2_ITEMREM=d2.D2_ITEMPV,
            )
        )
print("Remitos, PB7 y Facturas generados.")

Remitos, PB7 y Facturas generados.


# 6) Validaciones de reglas de negocio (post-carga, consultas rápidas)

In [20]:
with engine.begin() as conn:
    # 1) Remitos deben corresponder a ítems aprobados
    q = """
    SELECT COUNT(*) AS bad
    FROM SD2300 d
    WHERE d.D2_TIPODOC='50'
      AND NOT EXISTS (
        SELECT 1 FROM SC900 c9
        WHERE c9.C9_NUM = d.D2_PEDIDO
          AND c9.C9_ITEM = d.D2_ITEMPV
          AND c9.C9_STATUS = 'A'
      )
    """
    bad = pd.read_sql(q, conn)["bad"][0]
    print("Remitos sin aprobación:", bad)

    # 2) Traslados deben tener pares 501/008
    q2 = """
    SELECT COUNT(*) AS faltan_pares FROM (
      SELECT d3_xnrotra, d3_cod, d3_quant, d3_emissao,
             SUM(CASE WHEN d3_tm='501' THEN 1 ELSE 0 END) as salidas,
             SUM(CASE WHEN d3_tm='008' THEN 1 ELSE 0 END) as entradas
      FROM SD3300
      WHERE d3_xnrotra IS NOT NULL
      GROUP BY d3_xnrotra, d3_cod, d3_quant, d3_emissao
      HAVING SUM(CASE WHEN d3_tm='501' THEN 1 ELSE 0 END)=0
         OR  SUM(CASE WHEN d3_tm='008' THEN 1 ELSE 0 END)=0
    )
    """
    faltan = pd.read_sql(q2, conn)["faltan_pares"][0]
    print("Traslados con pares incompletos:", faltan)

    # 3) Facturas deben enlazar a algún remito (si tu regla lo exige)
    q3 = """
    SELECT COUNT(*) AS bad
    FROM SD2300 f
    WHERE f.D2_TIPODOC='01'
      AND (f.D2_REMITO IS NULL OR f.D2_SERIREM IS NULL)
    """
    bad2 = pd.read_sql(q3, conn)["bad"][0]
    print("Facturas sin remito enlazado:", bad2)

Remitos sin aprobación: 0
Traslados con pares incompletos: 0
Facturas sin remito enlazado: 0


# REPORTE DE TRASLADOS

SALIDA POR TRASLADO DE ALMACÉN (SD3 TM='008') que puedes pegar al final de tu fake.py. Usa los mismos modelos/engine que ya definiste y respeta:

- Paridad de traslado: siempre generar par 501 (origen) ↔ 008 (destino) con el mismo D3_XNROTRA, D3_COD, D3_QUANT, D3_EMISSAO.

- Reporte solicitado: solo filas TM='008', D3_XORIGEN <> ' ', D3_LOCAL NOT IN ('14','19','20'), D3_ESTORNO <> 'S' (lo simulamos no generando estornos).

- Productos/UM consistentes con SB1300.

- Fechas en rango.

## 1) Generador de traslados (crea pares 501→008 y aplica reglas del reporte)

In [21]:
# ===== SALIDA POR TRASLADO DE ALMACÉN: generador con reglas =====
from datetime import date

EXCLUDED_DEST = {"14", "19", "20"}  # no permitidos en el reporte
DEFAULT_FILIAL = "0001"

In [22]:
def generate_transfer_out_data(
    session: Session,
    start_date: date,
    end_date: date,
    pairs_per_day: int = 5,
    productos_whitelist=(
        "PA020211",
        "PA010001",
        "PA020121",
    ),  # azúcar 50kg, alcohol, azúcar 1kg
    origenes=("20", "90", "29", "63"),  # almacenes típicos de origen
    destinos=("29", "55", "63", "74", "90", "91"),  # destinos válidos
    usuario="ztransfer",
):
    """
    Genera traslados internos SD3 501->008 con reglas:
      - Cada 501 (salida origen) tiene su 008 (entrada destino).
      - 008: D3_XORIGEN <> ' ', D3_LOCAL NOT IN ('14','19','20').
      - Cantidades >0 y UM consistente con producto.
    """
    # Traer catálogos para validar CÓD -> UM
    prods = {p.B1_COD: p.B1_UM for p in session.query(SB1300).all()}
    alm_set = {a.ZH_ALM for a in session.query(SZH300).all()}

    # Filtrar productos a los que existan en catálogos
    productos = [p for p in productos_whitelist if p in prods]

    curr = start_date
    total_rows = 0

    while curr <= end_date:
        for k in range(pairs_per_day):
            cod = np.random.choice(productos)
            um = prods[cod]

            origen = np.random.choice(origenes)
            destino = np.random.choice(
                [
                    d
                    for d in destinos
                    if d in alm_set and d not in EXCLUDED_DEST and d != origen
                ]
            )

            # cantidad positiva coherente según UM
            if um == "LT":
                qty = float(np.random.uniform(5_000, 150_000))  # alcohol
            elif um in ("BL", "QQ"):
                qty = float(np.random.uniform(50, 1_200))  # azúcar (bolsas/qq)
            else:
                qty = float(np.random.uniform(50, 5_000))  # genérico

            # misma nota para el par
            nota = f"TR{curr.strftime('%Y%m%d')}{np.random.randint(100,999)}"

            # 1) Salida en origen (501)
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=qty + round(np.random.uniform(0.01, 0.99), 2),
                    D3_CUSTO1=round(np.random.uniform(0.5, 5.0), 4),
                    D3_LOCAL=origen,
                    D3_EMISSAO=curr,
                    D3_TM="501",
                    D3_XORIGEN="TR",  # marcado como traslado
                    D3_USUARIO=usuario,
                    D3_ESTORNO="N",
                )
            )

            # 2) Entrada en destino (008) — esta es la que muestra el reporte
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=qty + round(np.random.uniform(0.01, 0.99), 2),
                    D3_CUSTO1=round(np.random.uniform(0.5, 5.0), 4),
                    D3_LOCAL=destino,  # destino != ('14','19','20')
                    D3_EMISSAO=curr,
                    D3_TM="008",
                    D3_XORIGEN="TR",  # obligatorio para el reporte
                    D3_USUARIO=usuario,
                    D3_ESTORNO="N",
                )
            )
            total_rows += 2
        curr = curr + timedelta(days=1)

    return total_rows

In [23]:
# === Ejecutar generación y commit ===
with Session(engine) as s, s.begin():
    rows = generate_transfer_out_data(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 5),
        pairs_per_day=12,
        productos_whitelist=("PA020211", "PA010001", "PA020121"),
        origenes=("20", "90", "29", "63"),
        destinos=("29", "55", "63", "74", "90", "91"),
        usuario="bot_transfer",
    )
print(f"Traslados 501/008 generados: {rows} filas (incluye pares).")

Traslados 501/008 generados: 120 filas (incluye pares).


## 2) Validaciones de negocio específicas (post-carga)

In [24]:
with engine.begin() as conn:
    # A) Cada 008 tiene su 501 espejo
    v1 = pd.read_sql(
        """
        SELECT COUNT(*) AS faltan
        FROM SD3300 i
        WHERE i.D3_TM='008'
          AND NOT EXISTS (
            SELECT 1 FROM SD3300 o
            WHERE o.D3_TM='501'
              AND o.D3_XNROTRA = i.D3_XNROTRA
              AND o.D3_COD     = i.D3_COD
              AND o.D3_QUANT   = i.D3_QUANT
              AND o.D3_EMISSAO = i.D3_EMISSAO
          )
    """,
        conn,
    ).iloc[0, 0]
    print("008 sin 501 espejo:", v1)

    # B) 008 cumple XORIGEN y almacén no excluido
    v2 = pd.read_sql(
        """
        SELECT COUNT(*) AS invalidos
        FROM SD3300
        WHERE D3_TM='008'
          AND (NVL(TRIM(D3_XORIGEN),' ') = ' ' OR D3_LOCAL IN ('14','19','20'))
    """,
        conn,
    ).iloc[0, 0]
    print("008 con XORIGEN vacío o almacén excluido:", v2)

    # C) Productos válidos y UM consistente (join a SB1300)
    v3 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM SD3300 d
        LEFT JOIN SB1300 p ON p.B1_COD = d.D3_COD
        WHERE d.D3_TM='008'
          AND p.B1_COD IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("008 con producto inexistente en SB1300:", v3)

008 sin 501 espejo: 0
008 con XORIGEN vacío o almacén excluido: 10
008 con producto inexistente en SB1300: 0


## 3) Consulta de “reporte”

¿Qué productos y cantidades se movieron (salieron) de un almacén en una fecha y sucursal dadas, sin importar hacia dónde fueron trasladados?

In [25]:
with engine.begin() as conn:
    reporte = pd.read_sql(
        """
        SELECT
            SD3.D3_FILIAL AS "SUCURSAL",
            SD3.D3_EMISSAO AS "FECHA",
            SD3.D3_XNROTRA AS "NOTA",
            SB1.B1_DESC AS "PRODUCTO",
            SD3.D3_QUANT AS "BOLSA/PAQUETE",
            SD3.D3_QTSEGUM AS "QQ/L",
            SD3.D3_LOCAL AS "ALMACEN"
        FROM SD3300 SD3
        JOIN SB1300 SB1 ON SB1.B1_COD = SD3.D3_COD
        WHERE SD3.D3_TM = '008'
          AND NVL(TRIM(SD3.D3_XORIGEN),' ') <> ' '
          AND SD3.D3_LOCAL NOT IN ('14','19','20')
          -- AND SD3.D3_EMISSAO BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'
        ORDER BY SD3.D3_EMISSAO, SD3.D3_XNROTRA
    """,
        conn,
    )
reporte.head(10)

,sucursal,fecha,nota,producto,BOLSA/PAQUETE,QQ/L,almacen
0,0001,2020-06-01,TR20200601123,ALCOHOL HIDRATADO,146853.055,146853.905,91
1,0001,2020-06-01,TR20200601170,ALCOHOL HIDRATADO,59630.347,59631.277,90
2,0001,2020-06-01,TR20200601182,AZUCAR 50KG,1026.080,1026.510,63
3,0001,2020-06-01,TR20200601228,AZUCAR 1KG,246.562,247.372,29
4,0001,2020-06-01,TR20200601302,AZUCAR 50KG,427.817,427.997,90
5,0001,2020-06-01,TR20200601316,AZUCAR 50KG,1139.532,1139.902,91
6,0001,2020-06-01,TR20200601408,ALCOHOL HIDRATADO,56704.408,56704.498,90
7,0001,2020-06-01,TR20200601451,AZUCAR 1KG,857.358,857.708,74
8,0001,2020-06-01,TR20200601735,AZUCAR 50KG,86.363,87.213,55
9,0001,2020-06-01,TR20200601771,ALCOHOL HIDRATADO,56757.384,56758.184,55


In [26]:
# https://gemini.google.com/app/dc3c3f7956632b9b
# https://copilot.microsoft.com/chats/3MToYngQ9MYdUAcZHpm6k

# Reporte: 13 DATOS DE CHOFERv12.02

Supone que ya tienes creados el engine, Session y los modelos PB7300, SB1300, SC5300/SC6300, SD2300 (como en la versión anterior).

El generador crea entregas PB7 enlazadas a pedidos (SC6) y a remitos (SD2, D2_TIPODOC='50') para que las subconsultas del reporte funcionen tal cual

## 1) Generador de “Datos de Chofer” (PB7300) con reglas

In [27]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session

# ----- Parámetros/constantes útiles -----
DEFAULT_FILIAL = "0001"
PB7_STATUS_SALIDA = "S"  # 'S' = salida (despacho). ('E' entrada, 'F' anulado)

In [28]:
def _ensure_remito_for_pb7(
    session: Session,
    pedido: str,
    item: int,
    filial: str,
    produto: str,
    cliente: str,
    loja: str,
    local: str,
    qty: float,
    precio: float,
    emision: date,
) -> str:
    """
    Asegura un REMITO SD2300 (D2_TIPODOC='50') para enlazar con PB7,
    tal como lo espera el reporte.
    Retorna el número de remito (D2_DOC).
    """
    # ¿Ya existe un remito para (pedido,item,filial)?
    existing = (
        session.query(SD2300)
        .filter(
            SD2300.D2_TIPODOC == "50",
            SD2300.D2_PEDIDO == pedido,
            SD2300.D2_ITEMPV == item,
            SD2300.D2_FILIAL == filial,
        )
        .first()
    )
    if existing:
        return existing.D2_DOC

    # Crear un remito mínimo válido
    d2_doc = f"R{np.random.randint(10000,99999)}"
    session.add(
        SD2300(
            D2_TIPODOC="50",
            D2_DOC=d2_doc,
            D2_SERIE="001",
            D2_FILIAL=filial,
            D2_CLIENTE=cliente,
            D2_LOJA=loja,
            D2_COD=produto,
            D2_UM=session.query(SB1300).filter(SB1300.B1_COD == produto).first().B1_UM,
            D2_LOCAL=local,
            D2_QUANT=qty,
            D2_QTSEGUM=None,
            D2_PRCVEN=precio,
            D2_TOTAL=round(float(qty) * float(precio), 2),
            D2_TES="501",
            D2_CCUSTO=None,
            D2_EMISSAO=emision,
            D2_DTDIGIT=emision,
            D2_PEDIDO=pedido,
            D2_SEQUEN=item,
            D2_ITEMPV=item,
        )
    )
    session.flush()
    return d2_doc

In [29]:
def generate_driver_data(
    session: Session,
    start_date: date,
    end_date: date,
    deliveries_per_day: int = 12,
    allow_client_pickup_ratio: float = 0.15,  # % entregas sin chofer/placa (cliente retira)
    partials_ratio: float = 0.40,  # % entregas parciales (PB7_QTDE < C6_QTDVEN)
):
    """
    Genera PB7300 (Datos de Chofer / Balanza) respetando:
      - Enlace a SC6300 (pedido/ítem/filial) para subconsulta "Cant. Pedido".
      - Enlace a SD2300 (remito D2_TIPODOC='50') para subconsulta "REMITO".
      - Producto existente en SB1300 (pb7_produt=b1_cod).
      - Validaciones: PB7_QTDE <= C6_QTDVEN (suma por pedido-ítem no excede pedido).
      - Algunos casos sin chofer/placa (cliente retira).
    """
    # Traer universo de ítems de pedido aprobados/básicos
    sc6_items = session.query(SC6300).all()
    if not sc6_items:
        print("No hay SC6300; genera pedidos antes.")
        return 0

    productos = {p.B1_COD: p for p in session.query(SB1300).all()}
    total_pb7 = 0

    curr = start_date
    while curr <= end_date:
        # Seleccionar ítems aleatorios para despachar hoy
        todays = np.random.choice(
            sc6_items, size=min(deliveries_per_day, len(sc6_items)), replace=False
        )

        # Para controlar que la suma de despachos por (pedido, item) no exceda la cantidad pedida
        acumulado = {}

        for it in todays:
            key = (it.C6_NUM, it.C6_ITEM)
            cap = float(it.C6_QTDVEN)
            used = acumulado.get(key, 0.0)

            # Generar cantidad a entregar hoy
            if np.random.rand() < partials_ratio:
                # parcial
                remaining = max(cap - used, 0.0)
                if remaining <= 0:
                    continue
                qty = float(
                    np.random.uniform(remaining * 0.2, remaining)
                )  # de 20% a 100% del remanente
            else:
                # full o casi-full
                remaining = max(cap - used, 0.0)
                if remaining <= 0:
                    continue
                qty = remaining

            # Producto/UM
            prod = it.C6_PRODUTO
            if prod not in productos:
                continue
            um = productos[prod].B1_UM

            # Precio unit (si no está en SC6, tomar C6_PRCVEN)
            prc = float(it.C6_PRCVEN or 0.0)

            # PB7 "cliente retira" (sin chofer/placa) vs con chofer
            client_pickup = np.random.rand() < allow_client_pickup_ratio
            chofer = None if client_pickup else fake.name()
            placa = (
                None
                if client_pickup
                else f"{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}-{np.random.randint(100,999)}"
            )
            telefo = None if client_pickup else fake.phone_number()
            docume = None if client_pickup else str(np.random.randint(1000000, 9999999))

            # Remito (asegurado)
            _ = _ensure_remito_for_pb7(
                session=session,
                pedido=it.C6_NUM,
                item=it.C6_ITEM,
                filial=(it.C6_FILIAL or DEFAULT_FILIAL),
                produto=prod,
                cliente=it.C6_CLI,
                loja=it.C6_LOJA,
                local=(it.C6_LOCAL or "20"),
                qty=qty,
                precio=prc,
                emision=curr,
            )

            # PB7 row
            session.add(
                PB7300(
                    PB7_FILIAL=(it.C6_FILIAL or DEFAULT_FILIAL),
                    PB7_DATA=curr,
                    PB7_DSAIDA=curr,
                    PB7_PEDIDO=it.C6_NUM,
                    PB7_SEQUEN=it.C6_ITEM,
                    PB7_ITEMPV=it.C6_ITEM,
                    PB7_PRODUT=prod,
                    PB7_QTDE=qty,
                    PB7_STATUS=PB7_STATUS_SALIDA,
                    PB7_PBRUTO=(
                        (qty + np.random.uniform(0, 10))
                        if um not in ("LT", "KG")
                        else qty + np.random.uniform(50, 500)
                    ),
                    PB7_PTARA=np.random.uniform(0, 5),
                    PB7_PLACA=placa,
                    PB7_CHOFER=chofer,
                    PB7_TELEFO=telefo,
                    PB7_DOCUME=docume,
                    PB7_OBSERV=fake.sentence(nb_words=4),
                    PB7_XOBS=(
                        fake.sentence(nb_words=6) if np.random.rand() < 0.3 else None
                    ),
                    PB7_XZAFRA="2020",
                    PB7_XDESP=str(np.random.randint(10000, 99999)),
                    PB7_LOCENT=(it.C6_LOCDEST or it.C6_LOCAL or "20"),
                )
            )
            total_pb7 += 1
            acumulado[key] = used + qty

        curr += timedelta(days=1)

    return total_pb7

In [30]:
# === Ejecutar generación y commit ===
with Session(engine) as s, s.begin():
    total = generate_driver_data(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 5),
        deliveries_per_day=18,
        allow_client_pickup_ratio=0.20,  # 20% sin chofer/placa
        partials_ratio=0.50,  # 50% entregas parciales
    )
print(f"PB7300 generados: {total}")

PB7300 generados: 90


## 2) Validaciones de negocio (post-carga, estilo “sanity checks”)

In [31]:
with engine.begin() as conn:
    # 1) PB7 -> SB1 (producto debe existir)
    v1 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM PB7300 p
        LEFT JOIN SB1300 b ON b.B1_COD = p.PB7_PRODUT
        WHERE b.B1_COD IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin producto válido SB1300:", v1)

    # 2) PB7 -> SC6 (para subconsulta 'Cant. Pedido'): debe existir C6 por (num,item,filial)
    v2 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM PB7300 p
        LEFT JOIN SC6300 c6
          ON c6.C6_NUM = p.PB7_PEDIDO
         AND c6.C6_ITEM = p.PB7_ITEMPV
         AND NVL(c6.C6_FILIAL,'0001') = NVL(p.PB7_FILIAL,'0001')
        WHERE c6.C6_NUM IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin SC6 enlazado (pedido/ítem/filial):", v2)

    # 3) Suma de PB7_QTDE por (pedido,item) no debe exceder C6_QTDVEN
    v3 = pd.read_sql(
        """
        SELECT COUNT(*) AS excedidos
        FROM (
          SELECT p.PB7_PEDIDO, p.PB7_ITEMPV,
                 SUM(p.PB7_QTDE) as entregado,
                 MAX(c6.C6_QTDVEN) as pedido
          FROM PB7300 p
          JOIN SC6300 c6
            ON c6.C6_NUM = p.PB7_PEDIDO
           AND c6.C6_ITEM = p.PB7_ITEMPV
          GROUP BY p.PB7_PEDIDO, p.PB7_ITEMPV
        )
        WHERE entregado > pedido + 1e-6
    """,
        conn,
    ).iloc[0, 0]
    print("Pedido-ítem con entregas PB7 que exceden lo pedido:", v3)

    # 4) PB7 -> SD2 (remito) para subconsulta 'REMITO'
    v4 = pd.read_sql(
        """
        SELECT COUNT(*) AS faltan_remitos
        FROM PB7300 p
        LEFT JOIN SD2300 d
          ON d.D2_TIPODOC='50'
         AND d.D2_PEDIDO = p.PB7_PEDIDO
         AND d.D2_SEQUEN = p.PB7_SEQUEN
         AND NVL(d.D2_FILIAL,'0001') = NVL(p.PB7_FILIAL,'0001')
        WHERE d.D2_DOC IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin remito SD2(50) enlazado:", v4)

PB7 sin producto válido SB1300: 0
PB7 sin SC6 enlazado (pedido/ítem/filial): 0
Pedido-ítem con entregas PB7 que exceden lo pedido: 34
PB7 sin remito SD2(50) enlazado: 0


## 3) Consulta del reporte (igual a tu SQL)

In [32]:
with engine.begin() as conn:
    reporte_pb7 = pd.read_sql(
        """
        SELECT
           pb7.PB7_FILIAL AS "Sucursal",
           pb7.PB7_PRODUT AS "Producto",
           b1.B1_DESC     AS "Desc. Producto",
           pb7.PB7_QTDE   AS "Cantidad",
           (
             SELECT c6.C6_QTDVEN
             FROM SC6300 c6
             WHERE c6.C6_NUM    = pb7.PB7_PEDIDO
               AND NVL(c6.C6_FILIAL,'0001') = NVL(pb7.PB7_FILIAL,'0001')
               AND c6.C6_ITEM   = pb7.PB7_ITEMPV
               FETCH FIRST 1 ROWS ONLY
           ) AS "Cant. Pedido",
           (
             SELECT d.D2_DOC
             FROM SD2300 d
             WHERE d.D2_TIPODOC = '50'
               AND d.D2_PEDIDO  = pb7.PB7_PEDIDO
               AND d.D2_SEQUEN  = pb7.PB7_SEQUEN
               AND NVL(d.D2_FILIAL,'0001') = NVL(pb7.PB7_FILIAL,'0001')
               FETCH FIRST 1 ROWS ONLY
           ) AS "REMITO",
           pb7.PB7_DATA   AS "Fecha Ingreso Balanza",
           pb7.PB7_DSAIDA AS "Fecha Salida Balanza",
           pb7.PB7_PEDIDO AS "Pedido-Datos Cofer",
           pb7.PB7_SEQUEN AS "Secuencia",
           pb7.PB7_PLACA  AS "Placa",
           pb7.PB7_CHOFER AS "Chofer",
           pb7.PB7_TELEFO AS "Teléfono",
           pb7.PB7_DOCUME AS "DOCUMENTO",
           pb7.PB7_QTDE   AS "Cantidad de Entrega",
           pb7.PB7_STATUS AS "Entrada-Salida-Anulado",
           pb7.PB7_PBRUTO AS "P.BRUTO",
           pb7.PB7_PTARA  AS "TARA",
           pb7.PB7_OBSERV AS "OBS.",
           pb7.PB7_XOBS   AS "OBS. DE ENTREGA",
           pb7.PB7_XZAFRA AS "ZAFRA",
           pb7.PB7_XDESP  AS "N° DESPACHO",
           pb7.PB7_LOCENT AS "DESTINO"
        FROM PB7300 pb7
        JOIN SB1300 b1 ON b1.B1_COD = pb7.PB7_PRODUT
        --WHERE pb7.PB7_DATA BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'
        ORDER BY pb7.PB7_DATA, pb7.PB7_PEDIDO
    """,
        conn,
    )

reporte_pb7.head(10)

,Sucursal,Producto,Desc. Producto,Cantidad,Cant. Pedido,remito,Fecha Ingreso Balanza,Fecha Salida Balanza,Pedido-Datos Cofer,Secuencia,...,documento,Cantidad de Entrega,Entrada-Salida-Anulado,P.BRUTO,tara,OBS.,OBS. DE ENTREGA,zafra,N° DESPACHO,destino
0,0001,PA020211,AZUCAR 50KG,45.583,45.583,R48550,2020-03-01,2020-03-01,010500,1,...,6599315,45.583,S,51.826,4.823,None,None,2020,59170,74
1,0001,PA030001,BAGAZO,31.995,31.995,R96844,2020-03-01,2020-03-01,010501,2,...,5816779,31.995,S,34.552,1.446,None,None,2020,43764,63
2,0001,PA020211,AZUCAR 50KG,18.257,18.257,R33518,2020-03-01,2020-03-01,010502,1,...,9399090,18.257,S,22.323,0.522,None,None,2020,40368,29
3,0001,PA030001,BAGAZO,48.669,48.669,R19560,2020-03-01,2020-03-01,010503,1,...,7426733,48.669,S,57.469,1.385,None,None,2020,68123,74
4,0001,PA020121,AZUCAR 1KG,25.473,25.473,R60265,2020-03-01,2020-03-01,010504,1,...,4070804,25.473,S,35.015,3.692,None,None,2020,14539,63
5,0001,PA010001,ALCOHOL HIDRATADO,19121.066,19121.066,R60318,2020-03-01,2020-03-01,010505,1,...,6564705,19121.066,S,19130.932,0.397,None,None,2020,87686,91
6,0001,PA020121,AZUCAR 1KG,14.654,14.654,R44005,2020-03-01,2020-03-01,010505,2,...,5254636,14.654,S,19.756,0.728,None,None,2020,74305,90
7,0001,PA020211,AZUCAR 50KG,44.374,44.374,R39489,2020-03-01,2020-03-01,010506,1,...,9220109,44.374,S,51.288,4.285,None,None,2020,83714,29
8,0001,PA020121,AZUCAR 1KG,47.481,47.481,R31671,2020-03-01,2020-03-01,010507,1,...,5659355,47.481,S,53.835,2.250,None,None,2020,79659,29
9,0001,PA020211,AZUCAR 50KG,17.220,17.220,R86365,2020-03-01,2020-03-01,010508,2,...,6906841,17.220,S,22.953,2.887,None,None,2020,41738,90


- Generación realista de despachos con/ sin chofer (cliente retira) y entregas parciales.

- Enlaces garantizados:

    - PB7 → SB1 (producto)

    - PB7 → SC6 (subconsulta “Cant. Pedido” por pedido/ítem/filial)

    - PB7 → SD2(50) (subconsulta “REMITO”)

- Validaciones clave: productos válidos, relación con pedido, no exceder lo pedido en la suma de entregas, existencia de remito asociado.

# REPORTE: 04 SALIDAS_AZUCAR_ALCOHOL_v12.19 V6

- Parte A (SD2 + PB7): Remitos D2_TIPODOC='50' con TES de venta ('501') o traslado de almacén ('514'), enlazados a pedido/ítem/filial (SC6) y PB7300 para que funcionen las subconsultas de “Cant. Pedido” y “REMITO” y los campos de chofer/placa/peso.

- Parte B (SD3): Integraciones de traslados: TM='008' (entrada destino) con el par espejo TM='501' (salida origen) y mismas llaves de conciliación (D3_XNROTRA, D3_COD, D3_QUANT, D3_EMISSAO), cumpliendo D3_XORIGEN <> ' ' y excluir almacenes ('14','19','20').

SB1300, SA1300, SZG300, SC5300, SC6300, SD2300, PB7300, SD3300, CTT300, SZH300)

## 1) Generación de datos – Remitos + PB7 (venta TES=501 y traslado TES=514)

In [33]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session

DEFAULT_FILIAL = "0001"
EXC_ORDENES = set(
    [
        "127683",
        "128009",
        "128024",
        "128010",
        "128011",
        "127665",
        "127666",
        "127667",
        "127668",
    ]
)

In [34]:
def ensure_remito_50(
    session: Session,
    pedido: str,
    item: int,
    filial: str,
    produto: str,
    cliente: str,
    loja: str,
    local: str,
    qty: float,
    precio: float,
    emision: date,
    tes: str,
) -> str:
    """Garantiza un SD2 remito (TIPODOC=50) para pedido/ítem/filial, retorna D2_DOC."""
    r = (
        session.query(SD2300)
        .filter(
            SD2300.D2_TIPODOC == "50",
            SD2300.D2_PEDIDO == pedido,
            SD2300.D2_ITEMPV == item,
            SD2300.D2_FILIAL == filial,
        )
        .first()
    )
    if r:
        return r.D2_DOC
    d2_doc = f"R{np.random.randint(10000,99999)}"
    um = session.query(SB1300).filter(SB1300.B1_COD == produto).first().B1_UM
    session.add(
        SD2300(
            D2_TIPODOC="50",
            D2_DOC=d2_doc,
            D2_SERIE="001",
            D2_FILIAL=filial,
            D2_CLIENTE=cliente,
            D2_LOJA=loja,
            D2_COD=produto,
            D2_UM=um,
            D2_LOCAL=local,
            D2_QUANT=qty,
            D2_QTSEGUM=None,
            D2_PRCVEN=precio,
            D2_TOTAL=round(float(qty) * float(precio), 2),
            D2_TES=tes,  # 501 = venta, 514 = traslado de almacén
            D2_CCUSTO=np.random.choice(["100", "200", "300", "400"]),
            D2_CUSTO1=round(float(qty) * float(precio) * 0.7, 2),  # coste estimado
            D2_EMISSAO=emision,
            D2_DTDIGIT=emision,
            D2_PEDIDO=pedido,
            D2_SEQUEN=item,
            D2_ITEMPV=item,
        )
    )
    session.flush()
    return d2_doc

In [35]:
def generate_salidas_sd2_pb7(
    session: Session,
    start_date: date,
    end_date: date,
    remitos_por_dia: int = 18,
    ratio_tes_venta: float = 0.7,  # 70% venta (501), 30% traslado (514)
    allow_client_pickup_ratio: float = 0.15,
    partials_ratio: float = 0.45,
):
    """
    Genera Remitos SD2(50) y PB7300 asociados:
      - SD2.D2_TES en {'501','514'}
      - PB7 enlazado a SD2 y SC6
      - No incluye órdenes excluidas
      - Entregas parciales controladas para no exceder C6_QTDVEN
    """
    sc6_items = session.query(SC6300).all()
    if not sc6_items:
        print("No hay SC6300; genera pedidos primero.")
        return 0, 0

    prod_idx = {p.B1_COD: p for p in session.query(SB1300).all()}
    total_sd2, total_pb7 = 0, 0

    curr = start_date
    while curr <= end_date:
        # muestra del día
        today_items = np.random.choice(
            sc6_items, size=min(remitos_por_dia, len(sc6_items)), replace=False
        )

        # acumulado por (pedido,item) para no exceder lo pedido
        acumulado = {}

        for it in today_items:
            if it.C6_NUM in EXC_ORDENES:
                continue

            key = (it.C6_NUM, it.C6_ITEM)
            cap = float(it.C6_QTDVEN or 0.0)
            used = acumulado.get(key, 0.0)
            remaining = max(cap - used, 0.0)
            if remaining <= 0:
                continue

            # cantidad de la salida de hoy
            if np.random.rand() < partials_ratio:
                qty = float(np.random.uniform(max(remaining * 0.2, 0.01), remaining))
            else:
                qty = remaining

            prod = it.C6_PRODUTO
            if prod not in prod_idx or not prod.startswith("PA0"):
                continue

            # TES 501/514
            tes = "501" if (np.random.rand() < ratio_tes_venta) else "514"
            prc = float(it.C6_PRCVEN or 0.0)

            # asegurar remito
            d2_doc = ensure_remito_50(
                session=session,
                pedido=it.C6_NUM,
                item=it.C6_ITEM,
                filial=(it.C6_FILIAL or DEFAULT_FILIAL),
                produto=prod,
                cliente=it.C6_CLI,
                loja=it.C6_LOJA,
                local=(it.C6_LOCAL or "20"),
                qty=qty,
                precio=prc,
                emision=curr,
                tes=tes,
            )
            total_sd2 += 1

            # crear PB7
            client_pickup = np.random.rand() < allow_client_pickup_ratio
            chofer = None if client_pickup else fake.name()
            placa = (
                None
                if client_pickup
                else f"{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}-{np.random.randint(100,999)}"
            )
            telefo = None if client_pickup else fake.phone_number()
            docume = None if client_pickup else str(np.random.randint(1000000, 9999999))
            um = prod_idx[prod].B1_UM

            session.add(
                PB7300(
                    PB7_FILIAL=(it.C6_FILIAL or DEFAULT_FILIAL),
                    PB7_DATA=curr,
                    PB7_DSAIDA=curr,
                    PB7_PEDIDO=it.C6_NUM,
                    PB7_SEQUEN=it.C6_ITEM,
                    PB7_ITEMPV=it.C6_ITEM,
                    PB7_PRODUT=prod,
                    PB7_QTDE=qty,
                    PB7_STATUS="S",
                    PB7_PBRUTO=(
                        (qty + np.random.uniform(0, 10))
                        if um not in ("LT", "KG")
                        else qty + np.random.uniform(50, 500)
                    ),
                    PB7_PTARA=np.random.uniform(0, 5),
                    PB7_PLACA=placa,
                    PB7_CHOFER=chofer,
                    PB7_TELEFO=telefo,
                    PB7_DOCUME=docume,
                    PB7_OBSERV=fake.sentence(nb_words=4),
                    PB7_XOBS=(
                        fake.sentence(nb_words=6) if np.random.rand() < 0.25 else None
                    ),
                    PB7_XZAFRA="2020",
                    PB7_XDESP=str(np.random.randint(10000, 99999)),
                    PB7_LOCENT=(it.C6_LOCDEST or it.C6_LOCAL or "20"),
                )
            )
            total_pb7 += 1
            acumulado[key] = used + qty

        curr += timedelta(days=1)

    return total_sd2, total_pb7


In [36]:
# --- Ejecutar generación ---
with Session(engine) as s, s.begin():
    gen_sd2, gen_pb7 = generate_salidas_sd2_pb7(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 5),
        remitos_por_dia=24,
        ratio_tes_venta=0.7,  # 70% TES 501
        allow_client_pickup_ratio=0.2,  # 20% cliente retira (sin chofer/placa)
        partials_ratio=0.5,
    )
print(f"Generados SD2(50): {gen_sd2} | PB7: {gen_pb7}")

Generados SD2(50): 120 | PB7: 120


## 2) Generación de datos – Traslados SD3 (par 501→008, integraciones)

In [37]:
EXCLUDED_DEST = {"14", "19", "20"}

In [38]:
def generate_transfers_sd3(
    session: Session,
    start_date: date,
    end_date: date,
    pairs_per_day: int = 12,
    productos_whitelist=("PA020211", "PA010001", "PA020121"),
    origenes=("20", "90", "29", "63"),
    destinos=("29", "55", "63", "74", "90", "91"),
    usuario="bot_transfer",
):
    """Genera pares SD3 501→008 (integrados) cumpliendo las reglas del reporte."""
    prods = {p.B1_COD: p.B1_UM for p in session.query(SB1300).all()}
    alm_set = {a.ZH_ALM for a in session.query(SZH300).all()}
    productos = [p for p in productos_whitelist if p in prods]

    curr = start_date
    total_rows = 0
    while curr <= end_date:
        for _ in range(pairs_per_day):
            cod = np.random.choice(productos)
            um = prods[cod]
            origen = np.random.choice(origenes)
            destino = np.random.choice(
                [
                    d
                    for d in destinos
                    if d in alm_set and d not in EXCLUDED_DEST and d != origen
                ]
            )

            if um == "LT":
                qty = float(np.random.uniform(5_000, 150_000))
            elif um in ("BL", "QQ"):
                qty = float(np.random.uniform(50, 1_200))
            else:
                qty = float(np.random.uniform(50, 5_000))

            nota = f"TR{curr.strftime('%Y%m%d')}{np.random.randint(100,999)}"

            # Salida origen
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=None,
                    D3_CUSTO1=round(np.random.uniform(0.5, 5.0), 4),
                    D3_LOCAL=origen,
                    D3_EMISSAO=curr,
                    D3_TM="501",
                    D3_XORIGEN="TR",
                    D3_USUARIO=usuario,
                    D3_ESTORNO="N",
                )
            )
            # Entrada destino (aparece en el reporte)
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=None,
                    D3_CUSTO1=round(np.random.uniform(0.5, 5.0), 4),
                    D3_LOCAL=destino,
                    D3_EMISSAO=curr,
                    D3_TM="008",
                    D3_XORIGEN="TR",
                    D3_USUARIO=usuario,
                    D3_ESTORNO="N",
                )
            )
            total_rows += 2
        curr += timedelta(days=1)
    return total_rows

In [39]:
# --- Ejecutar generación ---
with Session(engine) as s, s.begin():
    gen_sd3 = generate_transfers_sd3(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 5),
        pairs_per_day=15,
        productos_whitelist=("PA020211", "PA010001", "PA020121"),
        origenes=("20", "90", "29", "63"),
        destinos=("29", "55", "63", "74", "90", "91"),
        usuario="bot_transfer",
    )
print(f"SD3 traslados generados (501+008): {gen_sd3}")

SD3 traslados generados (501+008): 150


## 3) Validaciones (sanity checks de negocio)

In [40]:
with engine.begin() as conn:
    # --- SD2 + PB7 ---
    # a) SD2(50) con producto 'PA0%' y join a SB1/SA1/SZG (catálogos válidos)
    v_sd2_prod = pd.read_sql(
        """
        SELECT COUNT(*) AS bad
        FROM SD2300 d
        LEFT JOIN SB1300 b ON b.B1_COD = d.D2_COD
        WHERE d.D2_TIPODOC='50'
          AND (b.B1_COD IS NULL OR d.D2_COD NOT LIKE 'PA0%')
    """,
        conn,
    ).iloc[0, 0]
    print("SD2(50) con producto inválido o no PA0%:", v_sd2_prod)

    # b) PB7 enlazado a SC6 (para subconsulta Cant. Pedido)
    v_pb7_sc6 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM PB7300 p
        LEFT JOIN SC6300 c6
          ON c6.C6_NUM  = p.PB7_PEDIDO
         AND c6.C6_ITEM = p.PB7_ITEMPV
         AND NVL(c6.C6_FILIAL,'0001') = NVL(p.PB7_FILIAL,'0001')
        WHERE c6.C6_NUM IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin SC6 enlazado:", v_pb7_sc6)

    # c) PB7 enlazado a SD2(50) por pedido/item/filial (para subconsulta REMITO)
    v_pb7_sd2 = pd.read_sql(
        """
        SELECT COUNT(*) AS faltan_remitos
        FROM PB7300 p
        LEFT JOIN SD2300 d
          ON d.D2_TIPODOC='50'
         AND d.D2_PEDIDO  = p.PB7_PEDIDO
         AND d.D2_SEQUEN  = p.PB7_SEQUEN
         AND NVL(d.D2_FILIAL,'0001') = NVL(p.PB7_FILIAL,'0001')
        WHERE d.D2_DOC IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin remito SD2(50):", v_pb7_sd2)

    # d) No exceder lo pedido (sum(PB7_QTDE) <= C6_QTDVEN por pedido/ítem)
    v_exceso = pd.read_sql(
        """
        SELECT COUNT(*) AS excedidos
        FROM (
          SELECT p.PB7_PEDIDO, p.PB7_ITEMPV,
                 SUM(p.PB7_QTDE) entregado, MAX(c6.C6_QTDVEN) pedido
          FROM PB7300 p
          JOIN SC6300 c6
            ON c6.C6_NUM = p.PB7_PEDIDO
           AND c6.C6_ITEM = p.PB7_ITEMPV
          GROUP BY p.PB7_PEDIDO, p.PB7_ITEMPV
        )
        WHERE entregado > pedido + 1e-6
    """,
        conn,
    ).iloc[0, 0]
    print("Pedido/ítem con entregas que exceden lo pedido:", v_exceso)

    # --- SD3 traslados ---
    # e) Cada 008 debe tener su 501 espejo
    v_pairs = pd.read_sql(
        """
        SELECT COUNT(*) AS faltan
        FROM SD3300 i
        WHERE i.D3_TM='008'
          AND NOT EXISTS (
            SELECT 1 FROM SD3300 o
            WHERE o.D3_TM='501'
              AND o.D3_XNROTRA = i.D3_XNROTRA
              AND o.D3_COD     = i.D3_COD
              AND o.D3_QUANT   = i.D3_QUANT
              AND o.D3_EMISSAO = i.D3_EMISSAO
          )
    """,
        conn,
    ).iloc[0, 0]
    print("SD3 008 sin 501 espejo:", v_pairs)

    # f) 008 cumpla XORIGEN y almacén no excluido
    v_008_rules = pd.read_sql(
        """
        SELECT COUNT(*) AS invalidos
        FROM SD3300
        WHERE D3_TM='008'
          AND (NVL(TRIM(D3_XORIGEN),' ')=' ' OR D3_LOCAL IN ('14','19','20'))
    """,
        conn,
    ).iloc[0, 0]
    print("SD3 008 con XORIGEN vacío o almacén excluido:", v_008_rules)

SD2(50) con producto inválido o no PA0%: 0
PB7 sin SC6 enlazado: 0
PB7 sin remito SD2(50): 0
Pedido/ítem con entregas que exceden lo pedido: 38
SD3 008 sin 501 espejo: 0
SD3 008 con XORIGEN vacío o almacén excluido: 10


## 4) Consulta final (idéntica estructura a tu SQL del reporte)

In [41]:
with engine.begin() as conn:
    reporte_salidas = pd.read_sql(
        """
        SELECT
           sd2.D2_FILIAL   AS "Sucursal",
           sd2.D2_DOC      AS "Nro Remito",
           sd2.D2_SERIE    AS "SERIE",
           sd2.D2_COD      AS "Producto",
           b1.B1_DESC      AS "Desc. Producto",
           sd2.D2_QUANT    AS "Cantidad",
           sd2.D2_QTSEGUM  AS "Cantidad qq",
           sd2.D2_CUSTO1   AS "Costo",
           sd2.D2_TES      AS "TES",
           NULL            AS "SISTEMA",
           sd2.D2_LOCAL    AS "Almacen Origen",
           (
             SELECT c6.C6_LOCDEST
             FROM SC6300 c6
             WHERE sd2.D2_PEDIDO = c6.C6_NUM
               AND sd2.D2_FILIAL = c6.C6_FILIAL
               AND sd2.D2_ITEMPV = c6.C6_ITEM
               FETCH FIRST 1 ROWS ONLY
           )               AS "Almacen Destino",
           sd2.D2_PEDIDO   AS "Pedido REMITO",
           sd2.D2_SEQUEN   AS "Secuencia REMITO",
           (
             SELECT c6.C6_QTDVEN
             FROM SC6300 c6
             WHERE sd2.D2_PEDIDO = c6.C6_NUM
               AND sd2.D2_FILIAL = c6.C6_FILIAL
               AND sd2.D2_ITEMPV = c6.C6_ITEM
               FETCH FIRST 1 ROWS ONLY
           )               AS "Cant. Pedido",
           sd2.D2_CLIENTE  AS "Cliente",
           sd2.D2_LOJA     AS "TIENDA",
           sa1.A1_NOME     AS "NOMBRE_CLI",
           sd2.D2_EMISSAO  AS "Emision REMITO",
           sd2.D2_DTDIGIT  AS "Digitacion REMITO",
           pb7.PB7_DATA    AS "Fecha Ingreso Balanza",
           pb7.PB7_DSAIDA  AS "Fecha Salida Balanza",
           pb7.PB7_PEDIDO  AS "Pedido-Datos Cofer",
           pb7.PB7_SEQUEN  AS "Secuencia",
           pb7.PB7_PLACA   AS "Placa",
           pb7.PB7_CHOFER  AS "Chofer",
           pb7.PB7_DOCUME  AS "DOCUMENTO",
           pb7.PB7_QTDE    AS "Cantidad de Entrega",
           pb7.PB7_STATUS  AS "Entrada-Salida-Anulado",
           pb7.PB7_PBRUTO  AS "P.BRUTO",
           pb7.PB7_PTARA   AS "TARA",
           pb7.PB7_OBSERV  AS "OBS.",
           pb7.PB7_XZAFRA  AS "ZAFRA",
           szg.ZG_DESC     AS "TIPO CLIENTE",
           pb7.PB7_XDESP   AS "N° DESPACHO",
           pb7.PB7_LOCENT  AS "DESTINO",
           (
             SELECT d.D2_SERIE
             FROM SD2300 d
             WHERE d.D2_REMITO = sd2.D2_DOC
               AND d.D2_SERIREM = sd2.D2_SERIE
               AND ROWNUM = 1
           )               AS "SERIE FAC",
           (
             SELECT d.D2_DOC
             FROM SD2300 d
             WHERE d.D2_REMITO = sd2.D2_DOC
               AND d.D2_SERIREM = sd2.D2_SERIE
               AND ROWNUM = 1
           )               AS "FACTURA",
           (
             SELECT d.D2_EMISSAO
             FROM SD2300 d
             WHERE d.D2_REMITO = sd2.D2_DOC
               AND d.D2_SERIREM = sd2.D2_SERIE
               AND ROWNUM = 1
           )               AS "FECHA FAC",
           (
             SELECT ctt.CTT_DESC01
             FROM CTT300 ctt
             WHERE ctt.CTT_CUSTO = sd2.D2_CCUSTO
           )               AS "SEGMENTO",
           (
             SELECT z.ZH_DESC
             FROM SZH300 z
             WHERE z.ZH_ALM = sd2.D2_LOCAL
           )               AS "MOVIL"
        FROM SD2300 sd2
        LEFT JOIN PB7300 pb7
          ON sd2.D2_PEDIDO = pb7.PB7_PEDIDO
         AND sd2.D2_FILIAL = pb7.PB7_FILIAL
         AND sd2.D2_SEQUEN = pb7.PB7_SEQUEN
         AND sd2.D2_ITEMPV = pb7.PB7_ITEMPV
         AND sd2.D2_COD    = pb7.PB7_PRODUT
        JOIN SB1300 b1 ON b1.B1_COD = sd2.D2_COD
        JOIN SA1300 sa1 ON sa1.A1_COD = sd2.D2_CLIENTE AND sa1.A1_LOJA = sd2.D2_LOJA
        JOIN SZG300 szg ON szg.ZG_CODIGO = sa1.A1_UTPCLI
        WHERE sd2.D2_TIPODOC = '50'
          AND sd2.D2_COD LIKE 'PA0%%'
        --  AND sd2.D2_DTDIGIT BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'
        UNION ALL
        SELECT
           sd3.D3_FILIAL  AS "Sucursal",
           sd3.D3_DOC     AS "Nro Remito",
           sd3.D3_XNROTRA AS "SERIE",
           sd3.D3_COD     AS "Producto",
           b1.B1_DESC     AS "Desc. Producto",
           sd3.D3_QUANT   AS "Cantidad",
           sd3.D3_QTSEGUM AS "Cantidad qq",
           sd3.D3_CUSTO1  AS "Costo",
           '501D'         AS "TES",
           sd3.D3_XORIGEN AS "SISTEMA",
           (
             SELECT DISTINCT t3.D3_LOCAL
             FROM SD3300 t3
             WHERE t3.D3_XNROTRA = sd3.D3_XNROTRA
               AND t3.D3_COD     = sd3.D3_COD
               AND t3.D3_QUANT   = sd3.D3_QUANT
               AND t3.D3_EMISSAO = sd3.D3_EMISSAO
               AND t3.D3_TM      = '501'
           )               AS "Almacen Origen",
           sd3.D3_LOCAL    AS "Almacen Destino",
           NULL            AS "Pedido REMITO",
           NULL            AS "Secuencia REMITO",
           sd3.D3_QUANT    AS "Cant. Pedido",
           NULL            AS "Cliente",
           NULL            AS "TIENDA",
           NULL            AS "NOMBRE_CLI",
           sd3.D3_EMISSAO  AS "Emision REMITO",
           sd3.D3_EMISSAO  AS "Digitacion REMITO",
           NULL            AS "Fecha Ingreso Balanza",
           NULL            AS "Fecha Salida Balanza",
           NULL            AS "Pedido-Datos Cofer",
           NULL            AS "Secuencia",
           NULL            AS "Placa",
           NULL            AS "Chofer",
           NULL            AS "DOCUMENTO",
           sd3.D3_QUANT    AS "Cantidad de Entrega",
           NULL            AS "Entrada-Salida-Anulado",
           NULL            AS "P.BRUTO",
           NULL            AS "TARA",
           NULL            AS "OBS.",
           NULL            AS "ZAFRA",
           NULL            AS "TIPO CLIENTE",
           NULL            AS "N° DESPACHO",
           NULL            AS "DESTINO",
           NULL            AS "SERIE FAC",
           NULL            AS "FACTURA",
           NULL            AS "FECHA FAC",
           NULL            AS "SEGMENTO",
           NULL            AS "MOVIL"
        FROM SD3300 sd3
        JOIN SB1300 b1 ON b1.B1_COD = sd3.D3_COD
        WHERE sd3.D3_ESTORNO <> 'S'
          AND NVL(TRIM(sd3.D3_XORIGEN),' ') <> ' '
          AND sd3.D3_TM = '008'
          AND sd3.D3_LOCAL NOT IN ('14','19','20')
        --  AND sd3.D3_EMISSAO BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'
        ORDER BY 1, 2
    """,
        conn,
    )

reporte_salidas.head(20)

,Sucursal,Nro Remito,serie,Producto,Desc. Producto,Cantidad,Cantidad qq,Costo,tes,sistema,...,OBS.,zafra,TIPO CLIENTE,N° DESPACHO,destino,SERIE FAC,factura,FECHA FAC,segmento,movil
0,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,None,2020,INTERESES,49467,20,001,F13498,2020-03-01,Mercado,Central
1,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,U segundo precisamente económica.,2020,INTERESES,87709,29,001,F13498,2020-03-01,Mercado,Central
2,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Salud acuerdo sólo estudios.,2020,INTERESES,95200,29,001,F13498,2020-03-01,Mercado,Central
3,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Siendo tendrá memoria.,2020,INTERESES,51384,29,001,F13498,2020-03-01,Mercado,Central
4,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Alguna imposible ocasión visita.,2020,INTERESES,77727,29,001,F13498,2020-03-01,Mercado,Central
5,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Acuerdo personas función hacía.,2020,INDUSTRIA,52789,74,001,F45937,2020-03-01,Mayorista,Mercado
6,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Sociedad ya ninguna necesidad otro.,2020,INDUSTRIA,35802,74,001,F45937,2020-03-01,Mayorista,Mercado
7,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Economía prensa.,2020,INDUSTRIA,70974,74,001,F45937,2020-03-01,Mayorista,Mercado
8,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Propio yo construcción.,2020,INDUSTRIA,79559,74,001,F45937,2020-03-01,Mayorista,Mercado
9,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Verdad forma usted.,2020,INDUSTRIA,30231,74,001,F45937,2020-03-01,Mayorista,Mercado


Genera:

- Remitos SD2300 (TIPODOC=50) con TES 501 (venta) y TES 514 (traslado de almacén), más PB7300 enlazado para datos de chofer/placa/pesos.

- Traslados SD3300 integrados con pares 501→008, cumpliendo XORIGEN y exclusiones de almacén.

Valida:

- Integridad SD2↔PB7↔SC6 (cantidades no exceden lo pedido, joins presentes).

- Pares SD3 008↔501 y reglas del reporte (xorigen, almacenes).

Consulta final: misma forma y campos que tu SQL de SALIDAS_AZÚCAR_ALCOHOL para contrastar con tu Excel.

# REPORTE: 02 FACTURAS_v12.23 Azúcar + Alcohol + Remito + Chofer

Respeta tu SQL (remitos SD2300 D2_TIPODOC='50' + sus facturas relacionadas SD2300 D2_TIPODOC='01' vía D2_REMITO / D2_SERIREM) y la unión con traslados integrados (SD3300 TM='008').

Asume que ya tienes: engine, Session y los modelos SB1300, SA1300, SZG300, SC5300, SC6300, SD2300, PB7300, SD3300, CTT300, SZH300 creados como en tus pasos previos.

## 1) Generación de remitos de venta (50) + facturas (01) y PB7

In [42]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session

DEFAULT_FILIAL = "0001"
EXC_ORDENES = set(
    [
        "127683",
        "128009",
        "128024",
        "128010",
        "128011",
        "127665",
        "127666",
        "127667",
        "127668",
    ]
)

In [43]:
def _ensure_remito(
    session: Session,
    it,
    qty: float,
    emision: date,
    tes: str = "501",
    local_default="20",
) -> str:
    """
    Crea (si no existe) el REMITO SD2300 con D2_TIPODOC='50' para (pedido,item,filial).
    Devuelve D2_DOC.
    """
    r = (
        session.query(SD2300)
        .filter(
            SD2300.D2_TIPODOC == "50",
            SD2300.D2_PEDIDO == it.C6_NUM,
            SD2300.D2_ITEMPV == it.C6_ITEM,
            SD2300.D2_FILIAL == (it.C6_FILIAL or DEFAULT_FILIAL),
        )
        .first()
    )
    if r:
        return r.D2_DOC

    prod = it.C6_PRODUTO
    sb1 = session.query(SB1300).filter(SB1300.B1_COD == prod).first()
    if not sb1:
        return None
    um = sb1.B1_UM
    d2_doc = f"R{np.random.randint(10000,99999)}"

    session.add(
        SD2300(
            D2_TIPODOC="50",
            D2_DOC=d2_doc,
            D2_SERIE="001",
            D2_FILIAL=(it.C6_FILIAL or DEFAULT_FILIAL),
            D2_CLIENTE=it.C6_CLI,
            D2_LOJA=it.C6_LOJA,
            D2_COD=prod,
            D2_UM=um,
            D2_LOCAL=(it.C6_LOCAL or local_default),
            D2_QUANT=qty,
            D2_QTSEGUM=None,
            D2_PRCVEN=float(it.C6_PRCVEN or 0),
            D2_TOTAL=round(float(qty) * float(it.C6_PRCVEN or 0), 2),
            D2_TES=tes,  # Venta
            D2_CCUSTO=np.random.choice(["100", "200", "300", "400"]),
            D2_EMISSAO=emision,
            D2_DTDIGIT=emision,
            D2_PEDIDO=it.C6_NUM,
            D2_SEQUEN=it.C6_ITEM,
            D2_ITEMPV=it.C6_ITEM,
        )
    )
    session.flush()
    return d2_doc

In [44]:
def _ensure_factura_from_remito(
    session: Session, remito_doc: str, remito_serie: str, it, emision: date
):
    """
    Crea la FACTURA SD2300 D2_TIPODOC='01' enlazada al remito (via D2_REMITO, D2_SERIREM).
    """
    # Ya existe factura enlazada a ese remito?
    f = (
        session.query(SD2300)
        .filter(
            SD2300.D2_TIPODOC == "01",
            SD2300.D2_REMITO == remito_doc,
            SD2300.D2_SERIREM == remito_serie,
            SD2300.D2_FILIAL == (it.C6_FILIAL or DEFAULT_FILIAL),
        )
        .first()
    )
    if f:
        return f.D2_DOC

    # Crear factura mínima (copiando lo esencial del ítem del remito/pedido)
    fac_doc = f"F{np.random.randint(100000,999999)}"
    sb1 = session.query(SB1300).filter(SB1300.B1_COD == it.C6_PRODUTO).first()
    um = sb1.B1_UM if sb1 else "UN"
    # Cantidad facturada alineada a lo de remito (para demo usamos C6_QTDVEN)
    qty = float(it.C6_QTDVEN or 0)

    session.add(
        SD2300(
            D2_TIPODOC="01",
            D2_DOC=fac_doc,
            D2_SERIE="A01",
            D2_FILIAL=(it.C6_FILIAL or DEFAULT_FILIAL),
            D2_CLIENTE=it.C6_CLI,
            D2_LOJA=it.C6_LOJA,
            D2_COD=it.C6_PRODUTO,
            D2_UM=um,
            D2_LOCAL=(it.C6_LOCAL or "20"),
            D2_QUANT=qty,
            D2_QTSEGUM=None,
            D2_PRCVEN=float(it.C6_PRCVEN or 0),
            D2_TOTAL=round(qty * float(it.C6_PRCVEN or 0), 2),
            D2_TES="501",
            D2_CCUSTO=np.random.choice(["100", "200", "300", "400"]),
            D2_EMISSAO=emision,  # fecha factura
            D2_DTDIGIT=emision,
            D2_PEDIDO=it.C6_NUM,
            D2_SEQUEN=it.C6_ITEM,
            D2_ITEMPV=it.C6_ITEM,
            D2_REMITO=remito_doc,  # vínculo remito -> factura
            D2_SERIREM=remito_serie,
        )
    )
    session.flush()
    return fac_doc

In [45]:
def generate_invoices_report_data(
    session: Session,
    start_date: date,
    end_date: date,
    docs_per_day: int = 22,
    partials_ratio: float = 0.45,
    allow_client_pickup_ratio: float = 0.18,
):
    """
    Genera: SD2 remitos (50) + SD2 facturas (01) y PB7 enlazado.
    Reglas:
      - Producto 'PA0%' válido en SB1300
      - No exceder lo pedido (sum(PB7_QTDE) <= C6_QTDVEN por pedido/ítem)
      - Factura enlazada a remito por (D2_REMITO, D2_SERIREM)
    """
    sc6_items = session.query(SC6300).all()
    if not sc6_items:
        print("No hay SC6300; carga pedidos primero.")
        return (0, 0, 0)

    prod_map = {p.B1_COD: p for p in session.query(SB1300).all()}
    total_remitos, total_facturas, total_pb7 = 0, 0, 0

    curr = start_date
    while curr <= end_date:
        today = np.random.choice(
            sc6_items, size=min(docs_per_day, len(sc6_items)), replace=False
        )
        acumulado = {}

        for it in today:
            if it.C6_NUM in EXC_ORDENES:  # excluir black-list
                continue
            prod = it.C6_PRODUTO
            if (prod not in prod_map) or (not prod.startswith("PA0")):
                continue

            cap = float(it.C6_QTDVEN or 0)
            used = acumulado.get((it.C6_NUM, it.C6_ITEM), 0.0)
            remaining = max(cap - used, 0.0)
            if remaining <= 0:
                continue

            # cantidad remito del día
            if np.random.rand() < partials_ratio:
                qty = float(np.random.uniform(max(remaining * 0.2, 0.01), remaining))
            else:
                qty = remaining

            # 1) Remito D2_TIPODOC=50 (venta TES=501)
            rdoc = _ensure_remito(session, it, qty, curr, tes="501")
            total_remitos += 1
            rserie = "001"

            # 2) PB7 (datos de chofer); opcional si el cliente retira
            client_pickup = np.random.rand() < allow_client_pickup_ratio
            sb1 = prod_map[prod]
            um = sb1.B1_UM
            session.add(
                PB7300(
                    PB7_FILIAL=(it.C6_FILIAL or DEFAULT_FILIAL),
                    PB7_DATA=curr,
                    PB7_DSAIDA=curr,
                    PB7_PEDIDO=it.C6_NUM,
                    PB7_SEQUEN=it.C6_ITEM,
                    PB7_ITEMPV=it.C6_ITEM,
                    PB7_PRODUT=prod,
                    PB7_QTDE=qty,
                    PB7_STATUS="S",
                    PB7_PBRUTO=(
                        (qty + np.random.uniform(0, 10))
                        if um not in ("LT", "KG")
                        else qty + np.random.uniform(50, 500)
                    ),
                    PB7_PTARA=np.random.uniform(0, 5),
                    PB7_PLACA=(
                        None
                        if client_pickup
                        else f"{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}{fake.random_uppercase_letter()}-{np.random.randint(100,999)}"
                    ),
                    PB7_CHOFER=None if client_pickup else fake.name(),
                    PB7_TELEFO=None if client_pickup else fake.phone_number(),
                    PB7_DOCUME=(
                        None
                        if client_pickup
                        else str(np.random.randint(1000000, 9999999))
                    ),
                    PB7_OBSERV=fake.sentence(nb_words=4),
                    PB7_XOBS=(
                        fake.sentence(nb_words=6) if np.random.rand() < 0.25 else None
                    ),
                    PB7_XZAFRA="2020",
                    PB7_XDESP=str(np.random.randint(10000, 99999)),
                    PB7_LOCENT=(it.C6_LOCDEST or it.C6_LOCAL or "20"),
                )
            )
            total_pb7 += 1
            acumulado[(it.C6_NUM, it.C6_ITEM)] = used + qty

            # 3) Factura D2_TIPODOC=01 enlazada al remito
            _ = _ensure_factura_from_remito(session, rdoc, rserie, it, curr)
            total_facturas += 1

        curr += timedelta(days=1)

    return (total_remitos, total_facturas, total_pb7)

In [46]:
# --- Ejecutar generación (REMITO 50 + FACTURA 01 + PB7) ---
with Session(engine) as s, s.begin():
    gen_r, gen_f, gen_pb7 = generate_invoices_report_data(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 5),
        docs_per_day=28,
        partials_ratio=0.5,
        allow_client_pickup_ratio=0.2,
    )
print(f"Generados: Remitos(50)={gen_r} | Facturas(01)={gen_f} | PB7={gen_pb7}")

Generados: Remitos(50)=140 | Facturas(01)=140 | PB7=140


## 2) (Opcional) Traslados integrados SD3 (para la segunda mitad del UNION ALL)

Si ya creaste pares SD3 501→008 en pasos anteriores, puedes saltar esta sección. Si no, aquí el generador:

In [47]:
EXCLUDED_DEST = {"14", "19", "20"}

In [48]:
def generate_transfers_sd3_for_invoice_report(
    session: Session,
    start_date: date,
    end_date: date,
    pairs_per_day: int = 10,
    productos_whitelist=("PA020211", "PA010001", "PA020121"),
    origenes=("20", "90", "29", "63"),
    destinos=("29", "55", "63", "74", "90", "91"),
    usuario="bot_transfer",
):
    prods = {p.B1_COD: p.B1_UM for p in session.query(SB1300).all()}
    alm_set = {a.ZH_ALM for a in session.query(SZH300).all()}
    productos = [p for p in productos_whitelist if p in prods]

    curr = start_date
    total = 0
    while curr <= end_date:
        for _ in range(pairs_per_day):
            cod = np.random.choice(productos)
            um = prods[cod]
            origen = np.random.choice(origenes)
            destino = np.random.choice(
                [
                    d
                    for d in destinos
                    if d in alm_set and d not in EXCLUDED_DEST and d != origen
                ]
            )

            if um == "LT":
                qty = float(np.random.uniform(5_000, 150_000))
            elif um in ("BL", "QQ"):
                qty = float(np.random.uniform(50, 1_200))
            else:
                qty = float(np.random.uniform(50, 5_000))

            nota = f"TR{curr.strftime('%Y%m%d')}{np.random.randint(100,999)}"

            # 501 salida
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=None,
                    D3_CUSTO1=None,
                    D3_LOCAL=origen,
                    D3_EMISSAO=curr,
                    D3_TM="501",
                    D3_XORIGEN="TR",
                    D3_USUARIO=usuario,
                )
            )
            # 008 entrada (aparece en reporte)
            session.add(
                SD3300(
                    D3_FILIAL=DEFAULT_FILIAL,
                    D3_DOC=None,
                    D3_XNROTRA=nota,
                    D3_COD=cod,
                    D3_UM=um,
                    D3_QUANT=qty,
                    D3_QTSEGUM=None,
                    D3_CUSTO1=None,
                    D3_LOCAL=destino,
                    D3_EMISSAO=curr,
                    D3_TM="008",
                    D3_XORIGEN="TR",
                    D3_USUARIO=usuario,
                )
            )
            total += 2
        curr += timedelta(days=1)
    return total

In [49]:
with Session(engine) as s, s.begin():
    gen_sd3 = generate_transfers_sd3_for_invoice_report(
        s,
        date(2020, 6, 1),
        date(2020, 6, 5),
        pairs_per_day=12,
        productos_whitelist=("PA020211", "PA010001", "PA020121"),
    )
print(f"SD3 traslados (501+008) generados: {gen_sd3}")

SD3 traslados (501+008) generados: 120


## 3) Validaciones de negocio

In [50]:
with engine.begin() as conn:
    # A) PB7 -> SC6 (para 'Cant. Pedido')
    v_pb7_sc6 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM PB7300 p
        LEFT JOIN SC6300 c6
          ON c6.C6_NUM  = p.PB7_PEDIDO
         AND c6.C6_ITEM = p.PB7_ITEMPV
         AND NVL(c6.C6_FILIAL,'0001') = NVL(p.PB7_FILIAL,'0001')
        WHERE c6.C6_NUM IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("PB7 sin SC6 enlazado:", v_pb7_sc6)

    # B) No exceder lo pedido (sum PB7 <= C6_QTDVEN)
    v_exceso = pd.read_sql(
        """
        SELECT COUNT(*) AS excedidos
        FROM (
          SELECT p.PB7_PEDIDO, p.PB7_ITEMPV,
                 SUM(p.PB7_QTDE) entregado, MAX(c6.C6_QTDVEN) pedido
          FROM PB7300 p
          JOIN SC6300 c6
            ON c6.C6_NUM = p.PB7_PEDIDO
           AND c6.C6_ITEM = p.PB7_ITEMPV
          GROUP BY p.PB7_PEDIDO, p.PB7_ITEMPV
        ) x
        WHERE x.entregado > x.pedido + 1e-6
    """,
        conn,
    ).iloc[0, 0]
    print("Entregas que exceden lo pedido:", v_exceso)

    # C) SD2(50) producto válido y 'PA0%'
    v_sd2_prod = pd.read_sql(
        """
        SELECT COUNT(*) AS bad
        FROM SD2300 d
        LEFT JOIN SB1300 b ON b.B1_COD = d.D2_COD
        WHERE d.D2_TIPODOC='50'
          AND (b.B1_COD IS NULL OR d.D2_COD NOT LIKE 'PA0%')
    """,
        conn,
    ).iloc[0, 0]
    print("SD2(50) con producto inválido/no PA0%:", v_sd2_prod)

    # D) SD2(50) debe tener su factura (01) enlazada por remito/serie
    v_sd2_fact = pd.read_sql(
        """
        SELECT COUNT(*) AS faltantes
        FROM SD2300 r
        WHERE r.D2_TIPODOC='50'
          AND NOT EXISTS (
            SELECT 1 FROM SD2300 f
            WHERE f.D2_TIPODOC='01'
              AND f.D2_REMITO = r.D2_DOC
              AND f.D2_SERIREM = r.D2_SERIE
          )
    """,
        conn,
    ).iloc[0, 0]
    print("Remitos (50) sin factura (01) enlazada:", v_sd2_fact)

    # E) SD3 008 tiene su 501 espejo y cumple reglas
    v_pairs = pd.read_sql(
        """
        SELECT COUNT(*) AS faltan
        FROM SD3300 i
        WHERE i.D3_TM='008'
          AND NOT EXISTS (
            SELECT 1 FROM SD3300 o
            WHERE o.D3_TM='501'
              AND o.D3_XNROTRA = i.D3_XNROTRA
              AND o.D3_COD     = i.D3_COD
              AND o.D3_QUANT   = i.D3_QUANT
              AND o.D3_EMISSAO = i.D3_EMISSAO
          )
    """,
        conn,
    ).iloc[0, 0]
    print("SD3 008 sin 501 espejo:", v_pairs)

    v_008_rules = pd.read_sql(
        """
        SELECT COUNT(*) AS invalidos
        FROM SD3300
        WHERE D3_TM='008'
          AND (NVL(TRIM(D3_XORIGEN),' ')=' ' OR D3_LOCAL IN ('14','19','20'))
    """,
        conn,
    ).iloc[0, 0]
    print("SD3 008 con XORIGEN vacío o almacén excluido:", v_008_rules)

PB7 sin SC6 enlazado: 0
Entregas que exceden lo pedido: 38
SD2(50) con producto inválido/no PA0%: 0
Remitos (50) sin factura (01) enlazada: 0
SD3 008 sin 501 espejo: 0
SD3 008 con XORIGEN vacío o almacén excluido: 10


## 4) Consulta final (misma estructura que tu SQL)

In [51]:
with engine.begin() as conn:
    facturas_reporte = pd.read_sql(
        """
        SELECT sd2.D2_FILIAL  AS "Sucursal",
               sd2.D2_DOC     AS "Nro Remito",
               sd2.D2_SERIE   AS "SERIE",
               sd2.D2_COD     AS "Producto",
               sb1.B1_DESC    AS "Desc. Producto",
               sd2.D2_QUANT   AS "Cantidad",
               sd2.D2_QTSEGUM AS "Cantidad qq",
               sd2.D2_CUSTO1  AS "Costo",
               sd2.D2_TES     AS "TES",
               NULL           AS "SISTEMA",
               sd2.D2_LOCAL   AS "Almacen Origen",
               (SELECT c6.C6_LOCDEST
                  FROM SC6300 c6
                 WHERE sd2.D2_PEDIDO = c6.C6_NUM
                   AND sd2.D2_FILIAL = c6.C6_FILIAL
                   AND sd2.D2_ITEMPV = c6.C6_ITEM
                   FETCH FIRST 1 ROWS ONLY) AS "Almacen Destino",
               sd2.D2_PEDIDO  AS "Pedido REMITO",
               sd2.D2_SEQUEN  AS "Secuencia REMITO",
               (SELECT c6.C6_QTDVEN
                  FROM SC6300 c6
                 WHERE sd2.D2_PEDIDO = c6.C6_NUM
                   AND sd2.D2_FILIAL = c6.C6_FILIAL
                   AND sd2.D2_ITEMPV = c6.C6_ITEM
                   FETCH FIRST 1 ROWS ONLY) AS "Cant. Pedido",
               sd2.D2_CLIENTE AS "Cliente",
               sd2.D2_LOJA    AS "TIENDA",
               sa1.A1_NOME    AS "NOMBRE_CLI",
               sd2.D2_EMISSAO AS "Emision REMITO",
               sd2.D2_DTDIGIT AS "Digitacion REMITO",
               pb7.PB7_DATA   AS "Fecha Ingreso Balanza",
               pb7.PB7_DSAIDA AS "Fecha Salida Balanza",
               pb7.PB7_PEDIDO AS "Pedido-Datos Cofer",
               pb7.PB7_SEQUEN AS "Secuencia",
               pb7.PB7_PLACA  AS "Placa",
               pb7.PB7_CHOFER AS "Chofer",
               pb7.PB7_DOCUME AS "DOCUMENTO",
               pb7.PB7_QTDE   AS "Cantidad de Entrega",
               pb7.PB7_STATUS AS "Entrada-Salida-Anulado",
               pb7.PB7_PBRUTO AS "P.BRUTO",
               pb7.PB7_PTARA  AS "TARA",
               pb7.PB7_OBSERV AS "OBS.",
               pb7.PB7_XZAFRA AS "ZAFRA",
               szg.ZG_DESC    AS "TIPO CLIENTE",
               pb7.PB7_XDESP  AS "N° DESPACHO",
               pb7.PB7_LOCENT AS "DESTINO",
               (SELECT d.D2_SERIE
                  FROM SD2300 d
                 WHERE d.D2_REMITO = sd2.D2_DOC
                   AND d.D2_SERIREM = sd2.D2_SERIE
                   AND ROWNUM = 1) AS "SERIE FAC",
               (SELECT d.D2_DOC
                  FROM SD2300 d
                 WHERE d.D2_REMITO = sd2.D2_DOC
                   AND d.D2_SERIREM = sd2.D2_SERIE
                   AND ROWNUM = 1) AS "FACTURA",
               (SELECT d.D2_EMISSAO
                  FROM SD2300 d
                 WHERE d.D2_REMITO = sd2.D2_DOC
                   AND d.D2_SERIREM = sd2.D2_SERIE
                   AND ROWNUM = 1) AS "FECHA FAC",
               (SELECT ctt.CTT_DESC01
                  FROM CTT300 ctt
                 WHERE ctt.CTT_CUSTO = sd2.D2_CCUSTO) AS "SEGMENTO",
               (SELECT z.ZH_DESC
                  FROM SZH300 z
                 WHERE z.ZH_ALM = sd2.D2_LOCAL) AS "MOVIL"
        FROM SD2300 sd2
        LEFT JOIN PB7300 pb7
               ON sd2.D2_PEDIDO = pb7.PB7_PEDIDO
              AND sd2.D2_FILIAL = pb7.PB7_FILIAL
              AND sd2.D2_SEQUEN = pb7.PB7_SEQUEN
              AND sd2.D2_ITEMPV = pb7.PB7_ITEMPV
              AND sd2.D2_COD    = pb7.PB7_PRODUT
        JOIN SB1300 sb1 ON sb1.B1_COD = sd2.D2_COD
        JOIN SA1300 sa1 ON sa1.A1_COD = sd2.D2_CLIENTE AND sa1.A1_LOJA = sd2.D2_LOJA
        JOIN SZG300 szg ON szg.ZG_CODIGO = sa1.A1_UTPCLI
        WHERE sd2.D2_TIPODOC = '50'
          AND sd2.D2_COD LIKE 'PA0%%'
        -- AND sd2.D2_DTDIGIT BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'

        UNION ALL

        SELECT sd3.D3_FILIAL  AS "Sucursal",
               sd3.D3_DOC     AS "Nro Remito",
               sd3.D3_XNROTRA AS "SERIE",
               sd3.D3_COD     AS "Producto",
               b1.B1_DESC     AS "Desc. Producto",
               sd3.D3_QUANT   AS "Cantidad",
               sd3.D3_QTSEGUM AS "Cantidad qq",
               sd3.D3_CUSTO1  AS "Costo",
               '501D'         AS "TES",
               sd3.D3_XORIGEN AS "SISTEMA",
               (SELECT DISTINCT t3.D3_LOCAL
                  FROM SD3300 t3
                 WHERE t3.D3_XNROTRA = sd3.D3_XNROTRA
                   AND t3.D3_COD     = sd3.D3_COD
                   AND t3.D3_QUANT   = sd3.D3_QUANT
                   AND t3.D3_EMISSAO = sd3.D3_EMISSAO
                   AND t3.D3_TM      = '501') AS "Almacen Origen",
               sd3.D3_LOCAL    AS "Almacen Destino",
               NULL            AS "Pedido REMITO",
               NULL            AS "Secuencia REMITO",
               sd3.D3_QUANT    AS "Cant. Pedido",
               NULL            AS "Cliente",
               NULL            AS "TIENDA",
               NULL            AS "NOMBRE_CLI",
               sd3.D3_EMISSAO  AS "Emision REMITO",
               sd3.D3_EMISSAO  AS "Digitacion REMITO",
               NULL            AS "Fecha Ingreso Balanza",
               NULL            AS "Fecha Salida Balanza",
               NULL            AS "Pedido-Datos Cofer",
               NULL            AS "Secuencia",
               NULL            AS "Placa",
               NULL            AS "Chofer",
               NULL            AS "DOCUMENTO",
               sd3.D3_QUANT    AS "Cantidad de Entrega",
               NULL            AS "Entrada-Salida-Anulado",
               NULL            AS "P.BRUTO",
               NULL            AS "TARA",
               NULL            AS "OBS.",
               NULL            AS "ZAFRA",
               NULL            AS "TIPO CLIENTE",
               NULL            AS "N° DESPACHO",
               NULL            AS "DESTINO",
               NULL            AS "SERIE FAC",
               NULL            AS "FACTURA",
               NULL            AS "FECHA FAC",
               NULL            AS "SEGMENTO",
               NULL            AS "MOVIL"
        FROM SD3300 sd3
        JOIN SB1300 b1 ON b1.B1_COD = sd3.D3_COD
        WHERE sd3.D3_ESTORNO <> 'S'
          AND NVL(TRIM(sd3.D3_XORIGEN),' ') <> ' '
          AND sd3.D3_TM = '008'
          AND sd3.D3_LOCAL NOT IN ('14','19','20')
        -- AND sd3.D3_EMISSAO BETWEEN DATE '2020-06-01' AND DATE '2020-06-05'
        ORDER BY 1, 2
    """,
        conn,
    )

facturas_reporte.head(20)

,Sucursal,Nro Remito,serie,Producto,Desc. Producto,Cantidad,Cantidad qq,Costo,tes,sistema,...,OBS.,zafra,TIPO CLIENTE,N° DESPACHO,destino,SERIE FAC,factura,FECHA FAC,segmento,movil
0,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Salud acuerdo sólo estudios.,2020,INTERESES,95200,29,001,F13498,2020-03-01,Mercado,Central
1,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Alguna imposible ocasión visita.,2020,INTERESES,77727,29,001,F13498,2020-03-01,Mercado,Central
2,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,None,2020,INTERESES,49467,20,001,F13498,2020-03-01,Mercado,Central
3,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Siendo tendrá memoria.,2020,INTERESES,51384,29,001,F13498,2020-03-01,Mercado,Central
4,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,U segundo precisamente económica.,2020,INTERESES,87709,29,001,F13498,2020-03-01,Mercado,Central
5,0001,R15699,001,PA030001,BAGAZO,27.735,NaN,2405.65,501,None,...,Barcelona arte quienes solamente.,2020,INTERESES,93449,29,001,F13498,2020-03-01,Mercado,Central
6,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,García debe sociales cuenta electoral.,2020,INDUSTRIA,60247,74,001,F45937,2020-03-01,Mayorista,Mercado
7,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Acuerdo personas función hacía.,2020,INDUSTRIA,52789,74,001,F45937,2020-03-01,Mayorista,Mercado
8,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Sí general pesetas torno.,2020,INDUSTRIA,55923,74,001,F45937,2020-03-01,Mayorista,Mercado
9,0001,R16363,001,PA030001,BAGAZO,34.740,NaN,3423.24,501,None,...,Economía prensa.,2020,INDUSTRIA,70974,74,001,F45937,2020-03-01,Mayorista,Mercado


Lo que cubre

- Genera:

    - Remitos (50) por venta TES=501, facturas (01) enlazadas por D2_REMITO/D2_SERIREM, y PB7300 asociado (chofer/placa/pesos).

    - Traslados SD3300 con pares 501→008 (si no los tenías ya).

- Valida: enlaces PB7↔SC6, no exceder lo pedido, SD2(50) con catálogos y factura (01) obligatoria enlazada, y reglas de SD3 (par espejo y XORIGEN/almacenes).

- Consulta final: misma forma que tu SQL para FACTURAS DE VENTA con la mitad de remitos/facturas y la mitad de traslados integrados.

# REPORTE: 01 PEDIDOS_VENTA_v12.01 (PEDIDOS / PEDIDOS_ALCOHOL / PEDIDOS_BAGAZO)

Supone que ya tienes engine, Session y los modelos SA1300 (clientes), SB1300 (productos), SZG300 (tipos de cliente), SC5300 (pedido cabecera) y SC6300 (pedido ítems), con la columna “borrado lógico” D_E_L_E_T_ y las columnas mencionadas en tus consultas.

## 1) Generación de datos fake (cabeceras SC5 y líneas SC6)

In [52]:
from datetime import date, timedelta
import numpy as np
import pandas as pd
from sqlalchemy.orm import Session

DEFAULT_FILIAL = "0001"

In [53]:
# --- helpers mínimos ---
def _ensure_tipo_clientes(session: Session):
    tipos = [
        ("01", "MERCADO"),
        ("02", "SUPERMERCADO"),
        ("03", "MAYORISTA"),
        ("04", "INDUSTRIA"),
        ("06", "INTERIOR"),
        ("08", "EXPORTACION"),
        ("20", "INSTITUCIONES"),
        ("26", "INTERESES"),
    ]
    existing = {t.ZG_CODIGO for t in session.query(SZG300).all()}
    for cod, desc in tipos:
        if cod not in existing:
            session.add(SZG300(ZG_CODIGO=cod, ZG_DESC=desc, D_E_L_E_T_=" "))
    session.flush()

In [54]:
def _ensure_clientes(session: Session, n=80):
    """
    Crea clientes variados con distintos tipos (ZG_CODIGO) y tiendas (A1_LOJA).
    """
    tipos = [t.ZG_CODIGO for t in session.query(SZG300).all()]
    existing = {(c.A1_COD, c.A1_LOJA) for c in session.query(SA1300).all()}
    for i in range(1, n + 1):
        cod = f"{i:06d}"
        loja = np.random.choice(["01", "02"])
        if (cod, loja) in existing:
            continue
        zg = np.random.choice(tipos)
        session.add(
            SA1300(
                A1_COD=cod,
                A1_LOJA=loja,
                A1_NOME=f"CLIENTE {i:03d}",
                A1_UTPCLI=zg,
                A1_CGC=f"{np.random.randint(1000000,99999999)}",
                D_E_L_E_T_=" ",
            )
        )
    session.flush()

In [55]:
def _ensure_productos(session: Session):
    """
    Crea un set compacto de productos representativos:
      - PA01%  (alcohol)
      - PA02%  (azúcar)
      - PA03%  (bagazo)
    """
    cat = {
        # Alcohol
        "PA010001": ("ALCOHOL HIDRATADO", "LT"),
        "PA010005": ("ALCOHOL ANHIDRO", "LT"),
        # Azúcar (referencia por si los usas más adelante)
        "PA020121": ("AZUCAR REFINADO 46 KG FRACC 1 KG", "QQ"),
        "PA020211": ("AZUCAR REFINADO 50 KG", "BL"),
        # Bagazo
        "PA030001": ("BAGAZO SECO", "KG"),
    }
    existing = {p.B1_COD for p in session.query(SB1300).all()}
    for cod, (desc, um) in cat.items():
        if cod not in existing:
            session.add(SB1300(B1_COD=cod, B1_DESC=desc, B1_UM=um, D_E_L_E_T_=" "))
    session.flush()

In [56]:
def _tipo_pedido():
    # Mapea a tu DECODE C5_XTIPO: 1..8 (usaremos las más comunes)
    return np.random.choice(
        list("155155268")
    )  # sesgo a NORMAL(5), VENTA LOCAL(1), EXPORT(2/8)

In [57]:
def _precio_sugerido(um, prod):
    if um == "LT":
        return float(np.random.uniform(0.4, 6.5))
    if um == "QQ":
        return float(np.random.uniform(150, 240))
    if um == "BL":
        return float(np.random.uniform(150, 230))
    if um == "KG":
        return float(np.random.uniform(0.05, 0.30))
    return float(np.random.uniform(1, 50))

In [58]:
def _tes_por_tipo_pedido(c5_xtipo):
    # aproximación: venta 501 por defecto, export 505/512 (solo ilustrativo)
    return {"1": "501", "2": "512", "8": "512"}.get(c5_xtipo, "501")

In [59]:
def generate_orders(
    session: Session,
    start_date: date,
    end_date: date,
    pedidos_por_dia=25,
    items_max_por_pedido=4,
    alcohol_ratio=0.35,
    bagazo_ratio=0.15,
):
    """
    Genera SC5300 (cabecera) y SC6300 (ítems) con:
      - Integridad: SC6.C6_NUM=SC5.C5_NUM, SC6->SB1, SC6->SA1, SA1->SZG
      - Fechas: C5_EMISSAO en rango
      - C6_QTDENT <= C6_QTDVEN
      - TES consistente con C5_XTIPO
    """
    clientes = session.query(SA1300).all()
    productos = session.query(SB1300).all()
    prod_alcohol = [p for p in productos if p.B1_COD.startswith("PA01")]
    prod_bagazo = [p for p in productos if p.B1_COD.startswith("PA03")]
    prod_otros = [
        p
        for p in productos
        if not (p.B1_COD.startswith("PA01") or p.B1_COD.startswith("PA03"))
    ]

    dia = start_date
    created5 = created6 = 0

    counter = 0
    while dia <= end_date:
        for _ in range(pedidos_por_dia):
            cli = np.random.choice(clientes)
            num = f"{10800+counter:06d}"  # secuencial, 6 dígitos
            counter += 1
            loja = cli.A1_LOJA
            xtipo = _tipo_pedido()
            moneda = np.random.choice(["1", "2"])  # 1: Bs, 2: USD p.ej.

            # SC5: cabecera
            sc5 = SC5300(
                C5_NUM=num,
                C5_CLIENTE=cli.A1_COD,
                C5_LOJACLI=loja,
                C5_EMISSAO=dia,
                C5_XTIPO=xtipo,
                C5_MOEDA=moneda,
                D_E_L_E_T_=" ",
            )
            session.add(sc5)
            created5 += 1

            n_items = np.random.randint(1, items_max_por_pedido + 1)
            for item_idx in range(1, n_items + 1):
                bucket = np.random.rand()
                if bucket < alcohol_ratio and prod_alcohol:
                    p = np.random.choice(prod_alcohol)
                elif bucket < alcohol_ratio + bagazo_ratio and prod_bagazo:
                    p = np.random.choice(prod_bagazo)
                else:
                    p = np.random.choice(prod_otros)

                # cantidades razonables por UM
                if p.B1_UM == "LT":
                    ctd = float(np.random.uniform(1_000, 120_000))
                elif p.B1_UM in ("QQ", "BL"):
                    ctd = float(np.random.uniform(10, 600))
                elif p.B1_UM == "KG":
                    ctd = float(np.random.uniform(500, 10_000))
                else:
                    ctd = float(np.random.uniform(5, 500))

                precio = _precio_sugerido(p.B1_UM, p.B1_COD)
                total = round(ctd * precio, 2)
                tes = _tes_por_tipo_pedido(xtipo)
                local = np.random.choice(["20", "29", "55", "63", "74", "90", "91"])

                # entregado (no superar lo pedido)
                entreg = max(0.0, ctd - np.random.uniform(0, ctd * 0.4))

                sc6 = SC6300(
                    C6_NUM=num,
                    C6_FILIAL=DEFAULT_FILIAL,
                    C6_ITEM=f"{item_idx:02d}",
                    C6_CLI=cli.A1_COD,
                    C6_LOJA=loja,
                    C6_PRODUTO=p.B1_COD,
                    C6_DESCRI=p.B1_DESC,
                    C6_UM=p.B1_UM,
                    C6_QTDVEN=ctd,
                    C6_PRCVEN=precio,
                    C6_VALOR=total,
                    C6_TES=tes,
                    C6_LOCAL=local,
                    C6_QTDENT=entreg,
                    C6_LOCDEST=np.random.choice(
                        ["20", "29", "55", "63", "74", "90", "91"]
                    ),
                    D_E_L_E_T_=" ",
                )
                session.add(sc6)
                created6 += 1

        dia += timedelta(days=1)

    session.flush()
    return created5, created6

In [60]:
# --- Semilla de datos para los reportes ---
with Session(engine) as s, s.begin():
    _ensure_tipo_clientes(s)
    _ensure_clientes(s, n=120)
    _ensure_productos(s)
    c5, c6 = generate_orders(
        s,
        start_date=date(2020, 3, 1),
        end_date=date(2020, 3, 10),
        pedidos_por_dia=30,
        items_max_por_pedido=4,
    )
print(f"Generados: SC5={c5} cabeceras, SC6={c6} ítems")

Generados: SC5=300 cabeceras, SC6=765 ítems


## 2) Validaciones de negocio (consistencia mínima para tus reportes)

In [61]:
with engine.begin() as conn:
    # A) Integridad SC6 -> SC5
    v_sc6_sc5 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM SC6300 i
        LEFT JOIN SC5300 h ON h.C5_NUM = i.C6_NUM
        WHERE h.C5_NUM IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("SC6 sin SC5:", v_sc6_sc5)

    # B) SC6 -> SB1 (producto válido)
    v_sc6_sb1 = pd.read_sql(
        """
        SELECT COUNT(*) AS invalidos
        FROM SC6300 i
        LEFT JOIN SB1300 p ON p.B1_COD = i.C6_PRODUTO
        WHERE p.B1_COD IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("SC6 con producto inexistente:", v_sc6_sb1)

    # C) SC6 -> SA1 + SZG (cliente y tipo cliente)
    v_sc6_sa1 = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM SC6300 i
        LEFT JOIN SA1300 c ON c.A1_COD=i.C6_CLI AND c.A1_LOJA=i.C6_LOJA
        WHERE c.A1_COD IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("SC6 sin cliente:", v_sc6_sa1)

    v_tipo_cli = pd.read_sql(
        """
        SELECT COUNT(*) AS huérfanos
        FROM SA1300 c
        LEFT JOIN SZG300 z ON z.ZG_CODIGO = c.A1_UTPCLI
        WHERE z.ZG_CODIGO IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("SA1 sin tipo cliente (SZG):", v_tipo_cli)

    # D) No exceder lo pedido ya lo respetamos al generar, pero verificamos:
    v_exceso = pd.read_sql(
        """
        SELECT COUNT(*) excede
        FROM SC6300
        WHERE C6_QTDENT > C6_QTDVEN + 1e-6
    """,
        conn,
    ).iloc[0, 0]
    print("Ítems con entregado > pedido:", v_exceso)

    # E) Borrado lógico: ningún registro '*' (para reportes)
    v_borrados = pd.read_sql(
        """
        SELECT SUM(CASE WHEN D_E_L_E_T_='*' THEN 1 ELSE 0 END) total
        FROM (
            SELECT D_E_L_E_T_ FROM SC5300
            UNION ALL SELECT D_E_L_E_T_ FROM SC6300
            UNION ALL SELECT D_E_L_E_T_ FROM SA1300
            UNION ALL SELECT D_E_L_E_T_ FROM SB1300
            UNION ALL SELECT D_E_L_E_T_ FROM SZG300
        )
    """,
        conn,
    ).iloc[0, 0]
    print("Registros marcados borrados (no deben salir):", v_borrados or 0)

SC6 sin SC5: 0
SC6 con producto inexistente: 0
SC6 sin cliente: 0
SA1 sin tipo cliente (SZG): 0
Ítems con entregado > pedido: 0
Registros marcados borrados (no deben salir): 0


## 3) Consultas para los tres reportes (idénticas a tu forma)

Ajusta el filtro de fechas en los WHERE si quieres limitar (aquí no forzamos fechas para que veas datos enseguida).

In [62]:
with engine.begin() as conn:
    # ---------------- PEDIDOS ----------------
    pedidos = pd.read_sql(
        """
        SELECT SC6.C6_NUM          AS "NRO PEDIDO",
               SC5.C5_CLIENTE      AS "COD. CLIENTE",
               SC5.C5_LOJACLI      AS "TIENDA CLI.",
               SA1.A1_NOME         AS "NOMBRE CLIENTE",
               SC5.C5_EMISSAO      AS "FECHA EMISION",
               CASE SC5.C5_XTIPO
                   WHEN '1' THEN 'VENTA LOCAL'
                   WHEN '2' THEN 'VENTA EXP. FCA'
                   WHEN '3' THEN 'VENTA EXP. FOV'
                   WHEN '4' THEN 'CONSIGNACION'
                   WHEN '5' THEN 'NORMAL'
                   WHEN '6' THEN 'DONACION'
                   WHEN '7' THEN 'TRASPASO'
                   WHEN '8' THEN 'MARKETING'
                   ELSE 'OTRO'
               END                 AS "TIPO PEDIDO",
               SC5.C5_MOEDA        AS "MONEDA",
               SC6.C6_ITEM         AS "ITEM",
               SC6.C6_PRODUTO      AS "COD. PRODUCTO",
               SB1.B1_DESC         AS "DESCRIPCION",
               SB1.B1_UM           AS "UM",
               SA1.A1_UTPCLI       AS "TP_CLIENTE",
               SZG.ZG_DESC         AS "TIPO CLIENTE",
               SC6.C6_QTDVEN       AS "CANT.",
               SC6.C6_PRCVEN       AS "PREC. VENT",
               SC6.C6_VALOR        AS "TOTAL",
               SC6.C6_TES          AS "TES",
               SC6.C6_LOCAL        AS "ALMACEN",
               SC6.C6_QTDENT       AS "CANT. ENTREGADA"
        FROM SC6300 SC6
        JOIN SC5300 SC5 ON SC6.C6_NUM = SC5.C5_NUM
        JOIN SA1300 SA1 ON SA1.A1_COD = SC6.C6_CLI AND SA1.A1_LOJA = SC6.C6_LOJA
        JOIN SZG300 SZG ON SZG.ZG_CODIGO = SA1.A1_UTPCLI
        JOIN SB1300 SB1 ON SB1.B1_COD = SC6.C6_PRODUTO
        WHERE SC6.D_E_L_E_T_ <> '*'
          AND SC5.D_E_L_E_T_ <> '*'
          AND SA1.D_E_L_E_T_ <> '*'
          AND SB1.D_E_L_E_T_ <> '*'
        ORDER BY SC6.C6_NUM
    """,
        conn,
    )
    print("PEDIDOS rows:", len(pedidos))

    # ------------- PEDIDOS_ALCOHOL (PA01%) -------------
    pedidos_alcohol = pd.read_sql(
        """
        SELECT SC6.C6_NUM          AS "NRO PEDIDO",
               SC5.C5_CLIENTE      AS "COD. CLIENTE",
               SC5.C5_LOJACLI      AS "TIENDA CLI.",
               SA1.A1_NOME         AS "NOMBRE CLIENTE",
               SC5.C5_EMISSAO      AS "FECHA EMISION",
               CASE SC5.C5_XTIPO
                   WHEN '1' THEN 'VENTA LOCAL'
                   WHEN '2' THEN 'VENTA EXP. FCA'
                   WHEN '3' THEN 'VENTA EXP. FOV'
                   WHEN '4' THEN 'CONSIGNACION'
                   WHEN '5' THEN 'NORMAL'
                   WHEN '6' THEN 'DONACION'
                   WHEN '7' THEN 'TRASPASO'
                   WHEN '8' THEN 'MARKETING'
                   ELSE 'OTRO'
               END                 AS "TIPO PEDIDO",
               SC5.C5_MOEDA        AS "MONEDA",
               SC6.C6_ITEM         AS "ITEM",
               SC6.C6_PRODUTO      AS "COD. PRODUCTO",
               SC6.C6_DESCRI       AS "DESCRIPCION",
               SC6.C6_UM           AS "UM",
               SA1.A1_UTPCLI       AS "TP_CLIENTE",
               SZG.ZG_DESC         AS "TIPO CLIENTE",
               SC6.C6_QTDVEN       AS "CANT.",
               SC6.C6_PRCVEN       AS "PREC. VENT",
               SC6.C6_VALOR        AS "TOTAL",
               SC6.C6_TES          AS "TES",
               SC6.C6_LOCAL        AS "ALMACEN",
               SC6.C6_QTDENT       AS "CANT. ENTREGADA"
        FROM SC6300 SC6
        JOIN SC5300 SC5 ON SC6.C6_NUM = SC5.C5_NUM
        JOIN SA1300 SA1 ON SA1.A1_COD = SC6.C6_CLI AND SA1.A1_LOJA = SC6.C6_LOJA
        JOIN SZG300 SZG ON SZG.ZG_CODIGO = SA1.A1_UTPCLI
        WHERE SC6.D_E_L_E_T_ <> '*'
          AND SC5.D_E_L_E_T_ <> '*'
          AND SA1.D_E_L_E_T_ <> '*'
          AND SC6.C6_PRODUTO LIKE 'PA01%%'
        ORDER BY SC6.C6_NUM
    """,
        conn,
    )
    print("PEDIDOS_ALCOHOL rows:", len(pedidos_alcohol))

    # ------------- PEDIDOS_BAGAZO (PA03%) -------------
    pedidos_bagazo = pd.read_sql(
        """
        SELECT SC6.C6_NUM          AS "NRO PEDIDO",
               SC5.C5_CLIENTE      AS "COD. CLIENTE",
               SC5.C5_LOJACLI      AS "TIENDA CLI.",
               SA1.A1_NOME         AS "NOMBRE CLIENTE",
               SC5.C5_EMISSAO      AS "FECHA EMISION",
               CASE SC5.C5_XTIPO
                   WHEN '1' THEN 'VENTA LOCAL'
                   WHEN '2' THEN 'VENTA EXP. FCA'
                   WHEN '3' THEN 'VENTA EXP. FOV'
                   WHEN '4' THEN 'CONSIGNACION'
                   WHEN '5' THEN 'NORMAL'
                   WHEN '6' THEN 'DONACION'
                   WHEN '7' THEN 'TRASPASO'
                   WHEN '8' THEN 'MARKETING'
                   ELSE 'OTRO'
               END                 AS "TIPO PEDIDO",
               SC5.C5_MOEDA        AS "MONEDA",
               SC6.C6_ITEM         AS "ITEM",
               SC6.C6_PRODUTO      AS "COD. PRODUCTO",
               SC6.C6_DESCRI       AS "DESCRIPCION",
               SC6.C6_UM           AS "UM",
               SA1.A1_UTPCLI       AS "TP_CLIENTE",
               SZG.ZG_DESC         AS "TIPO CLIENTE",
               SC6.C6_QTDVEN       AS "CANT.",
               SC6.C6_PRCVEN       AS "PREC. VENT",
               SC6.C6_VALOR        AS "TOTAL",
               SC6.C6_TES          AS "TES",
               SC6.C6_LOCAL        AS "ALMACEN",
               SC6.C6_QTDENT       AS "CANT. ENTREGADA"
        FROM SC6300 SC6
        JOIN SC5300 SC5 ON SC6.C6_NUM = SC5.C5_NUM
        JOIN SA1300 SA1 ON SA1.A1_COD = SC6.C6_CLI AND SA1.A1_LOJA = SC6.C6_LOJA
        JOIN SZG300 SZG ON SZG.ZG_CODIGO = SA1.A1_UTPCLI
        WHERE SC6.D_E_L_E_T_ <> '*'
          AND SC5.D_E_L_E_T_ <> '*'
          AND SA1.D_E_L_E_T_ <> '*'
          AND SC6.C6_PRODUTO LIKE 'PA03%%'
        ORDER BY SC6.C6_NUM
    """,
        conn,
    )
    print("PEDIDOS_BAGAZO rows:", len(pedidos_bagazo))

PEDIDOS rows: 803
PEDIDOS_ALCOHOL rows: 275
PEDIDOS_BAGAZO rows: 141


Qué reglas se respetan aquí

- Integridad:

    - SC6.C6_NUM = SC5.C5_NUM (líneas ↔ cabecera).

    - SC6.C6_PRODUTO = SB1.B1_COD (producto existe en catálogo).

    - SC6.C6_CLI = SA1.A1_COD y SC6.C6_LOJA = SA1.A1_LOJA (cliente válido).

    - SA1.A1_UTPCLI = SZG.ZG_CODIGO (tipo cliente).

- Negocio:

    - C6_QTDENT <= C6_QTDVEN.

    - TES coherente con C5_XTIPO (heurística: 501 ventas locales/normal; 512 exportación).

    - D_E_L_E_T_ <> '*' en todos.

- Filtros de cada reporte:

    - General: sin filtro de familia de producto.

    - Alcohol: C6_PRODUTO LIKE 'PA01%'.

    - Bagazo: C6_PRODUTO LIKE 'PA03%'.

# REPORTE: 01 PEDIDOS_VENTA_v12.01 ALCOHOL y BAGAZO

- INGRESOS_AZUCAR (producción de azúcar; PA02%)

- INGRESO_ALCOHOL (producción de alcohol; PA01%)

Respeta tus reglas de negocio:

D3_TM = '001' (ingreso), D3_ESTORNO <> 'S' (no revertido), D_E_L_E_T_ = ' ', D3_COD → SB1.B1_COD, y tipos/UM consistentes (PA01% → LT; PA02% → QQ o BL).

Supone que ya tienes engine, Session y los modelos SD3300 y SB1300 (y su columna de borrado lógico).

## 1) Generación de datos fake (SD3300) para ambos reportes

In [63]:
from datetime import date, timedelta
import numpy as np
from sqlalchemy.orm import Session

USUARIOS_INGRESO = ["adaza", "rmontero", "jsuarez", "mmamani"]

In [64]:
def _ensure_catalogo_ingresos(session: Session):
    """
    Asegura un catálogo mínimo de productos coherente con los reportes:
      - Alcohol (PA01%)  -> UM = LT
      - Azúcar  (PA02%)  -> UM = QQ o BL (ambas existen en tus reportes)
    """
    cat = {
        # Alcohol
        "PA010001": ("ALCOHOL HIDRATADO", "LT"),
        "PA010005": ("ALCOHOL ANHIDRO", "LT"),
        # Azúcar
        "PA020121": ("AZUCAR REFINADO 46 KG FRACC 1 KG", "QQ"),
        "PA020211": ("AZUCAR REFINADO 50 KG", "BL"),
        "PA020311": ("AZUCAR REFINADO 46 KG", "QQ"),
        "PA020411": ("AZUCAR REFINADO 50 KG FRACC 5 KG", "BL"),
    }
    existing = {p.B1_COD for p in session.query(SB1300).all()}
    for cod, (desc, um) in cat.items():
        if cod not in existing:
            session.add(SB1300(B1_COD=cod, B1_DESC=desc, B1_UM=um, D_E_L_E_T_=" "))
    session.flush()

In [65]:
def _doc_num(fecha: date) -> str:
    # Para parecerse a tus muestras (yyyyMMdd o con sufijo). Sencillo y único por día.
    return fecha.strftime("%Y%m%d")

In [66]:
def _rand_cost(total_qty: float, um: str) -> float:
    """
    Costo total (D3_CUSTO1) aproximado para la línea completa (no unitario).
    Tomamos rangos similares a tus ejemplos.
    """
    if um == "LT":
        # alcohol: ~3.0 a 3.5 Bs/L por ejemplo
        unit = np.random.uniform(2.8, 3.4)
    elif um == "QQ":
        # azúcar qq: ~90–110 Bs/qq
        unit = np.random.uniform(85, 115)
    elif um == "BL":
        # azúcar bolsa/saco: ~90–110 Bs (referencial)
        unit = np.random.uniform(90, 120)
    else:
        unit = np.random.uniform(10, 50)
    return round(total_qty * unit, 2)

In [67]:
def _qty_for_um(um: str) -> float:
    """
    Cantidades razonables por UM, en rangos similares a tus reportes.
    """
    if um == "LT":  # alcohol
        return float(np.random.uniform(50_000, 230_000))
    if um == "QQ":  # azúcar (quintal)
        return float(np.random.uniform(300, 1_500))
    if um == "BL":  # azúcar (bolsas/sacos)
        return float(np.random.uniform(500, 7_000))
    return float(np.random.uniform(10, 500))

In [68]:
def _qty_secondary(um: str, qty: float) -> float:
    """
    D3_QTSEGUM (segunda unidad):
      - Alcohol: "CANTIDAD L" = igual a LT
      - Azúcar: "CANTIDAD QQ" ≈ QQ si UM=QQ; si UM=BL podemos aproximar a QQ (1 BL ~ 1.088 QQ p.ej.)
    """
    if um == "LT":
        return qty
    if um == "QQ":
        return qty
    if um == "BL":
        # conversión ligera a qq, si quieres afinar cambia el factor
        return round(qty * np.random.uniform(1.05, 1.15), 2)
    return qty

In [69]:
def _make_sd3_row(fecha: date, prod, usuario: str):
    """
    Crea una fila SD3300 coherente con ingreso de producción (TM=001, no estorno),
    enlazada al catálogo (SB1300).
    """
    um = prod.B1_UM
    qty = _qty_for_um(um)
    qtseg = _qty_secondary(um, qty)
    costo = _rand_cost(qty, um)

    return SD3300(
        D_E_L_E_T_=" ",
        D3_TM="001",  # ingreso de producción
        D3_ESTORNO=" ",  # no revertido
        D3_COD=prod.B1_COD,
        D3_UM=um,
        D3_QUANT=qty,
        D3_QTSEGUM=qtseg,
        D3_DOC=_doc_num(fecha),
        D3_EMISSAO=fecha,
        D3_CUSTO1=costo,
        D3_USUARIO=usuario,
        # Si manejas D3_LOCAL u otros, puedes setear un default de planta, p.ej. '01'
        D3_LOCAL="20",
    )

In [70]:
def seed_ingresos_sd3(
    session: Session,
    start_date: date,
    end_date: date,
    registros_por_dia_azucar=6,
    registros_por_dia_alcohol=3,
):
    """
    Genera ingresos diarios en SD3300:
      - Azúcar (PA02%) con UM QQ/BL
      - Alcohol (PA01%) con UM LT
    """
    _ensure_catalogo_ingresos(session)

    # separa por prefijo
    productos = session.query(SB1300).filter(SB1300.D_E_L_E_T_ == " ").all()
    prod_azucar = [p for p in productos if p.B1_COD.startswith("PA02")]
    prod_alcohol = [p for p in productos if p.B1_COD.startswith("PA01")]

    dia = start_date
    created = 0
    while dia <= end_date:
        # ingresos azúcar
        for _ in range(registros_por_dia_azucar):
            p = np.random.choice(prod_azucar)
            u = np.random.choice(USUARIOS_INGRESO)
            session.add(_make_sd3_row(dia, p, u))
            created += 1
        # ingresos alcohol
        for _ in range(registros_por_dia_alcohol):
            p = np.random.choice(prod_alcohol)
            u = np.random.choice(USUARIOS_INGRESO)
            session.add(_make_sd3_row(dia, p, u))
            created += 1
        dia += timedelta(days=1)

    session.flush()
    return created

In [71]:
# --- Ejecutar semilla de ingresos ---
from datetime import date

with Session(engine) as s, s.begin():
    n = seed_ingresos_sd3(
        s,
        start_date=date(2020, 6, 1),
        end_date=date(2020, 6, 7),
        registros_por_dia_azucar=4,  # ajusta a tu gusto
        registros_por_dia_alcohol=2,
    )
print(f"Ingresos SD3 generados: {n}")

Ingresos SD3 generados: 42


## 2) Validaciones de negocio (consistencia y filtros de tus reportes)

In [72]:
import pandas as pd

with engine.begin() as conn:
    # A) SD3 -> SB1 (producto debe existir)
    v_sd3_sb1 = pd.read_sql(
        """
        SELECT COUNT(*) AS invalidos
        FROM SD3300 s
        LEFT JOIN SB1300 p ON p.B1_COD = s.D3_COD AND p.D_E_L_E_T_=' '
        WHERE s.D_E_L_E_T_=' ' AND p.B1_COD IS NULL
    """,
        conn,
    ).iloc[0, 0]
    print("SD3 con producto inexistente:", v_sd3_sb1)

    # B) Ingreso válido: TM=001 y no estornado
    v_tm = (
        pd.read_sql(
            """
        SELECT SUM(CASE WHEN D3_TM='001' THEN 0 ELSE 1 END) AS tm_inval,
               SUM(CASE WHEN NVL(D3_ESTORNO,' ')='S' THEN 1 ELSE 0 END) AS estornos
        FROM SD3300
        WHERE D_E_L_E_T_=' '
    """,
            conn,
        )
        .iloc[0]
        .to_dict()
    )
    print(
        "TM inválidos:",
        v_tm["tm_inval"],
        " / Estornos marcados:",
        v_tm["ESTORNOS"] if "ESTORNOS" in v_tm else v_tm["estornos"],
    )

    # C) Unidades coherentes: PA01% => LT; PA02% => QQ o BL
    v_um = (
        pd.read_sql(
            """
        SELECT SUM(CASE WHEN D3_COD LIKE 'PA01%' AND D3_UM <> 'LT' THEN 1 ELSE 0 END) AS alcohol_um_bad,
               SUM(CASE WHEN D3_COD LIKE 'PA02%' AND D3_UM NOT IN ('QQ','BL') THEN 1 ELSE 0 END) AS azucar_um_bad
        FROM SD3300
        WHERE D_E_L_E_T_=' '
    """,
            conn,
        )
        .iloc[0]
        .to_dict()
    )
    print("UM inconsistentes:", v_um)

    # D) Cantidades positivas
    v_qty = pd.read_sql(
        """
        SELECT SUM(CASE WHEN D3_QUANT <= 0 THEN 1 ELSE 0 END) AS qty_bad
        FROM SD3300
        WHERE D_E_L_E_T_=' '
    """,
        conn,
    ).iloc[0, 0]
    print("Filas con cantidad <= 0:", v_qty)

SD3 con producto inexistente: 0
TM inválidos: 410  / Estornos marcados: 0
UM inconsistentes: {'alcohol_um_bad': 0, 'azucar_um_bad': 0}
Filas con cantidad <= 0: 0


## 3) Consultas que replican tus reportes (idénticas a tu estilo)

In [73]:
# --------- INGRESOS_AZUCAR (PA02%) ----------
with engine.begin() as conn:
    ingresos_azucar = pd.read_sql(
        """
        SELECT s.D3_COD                              AS "CODIGO",
               p.B1_DESC                             AS "DESCRIPCION",
               s.D3_UM                               AS "UM",
               s.D3_QUANT                            AS "CANTIDAD",
               s.D3_QTSEGUM                          AS "CANTIDAD QQ",
               s.D3_DOC                              AS "DOCUMENTO",
               s.D3_EMISSAO                          AS "EMISION",
               s.D3_CUSTO1                           AS "COSTO_BS",
               s.D3_USUARIO                          AS "USUARIO_INGRESO"
        FROM SD3300 s
        JOIN SB1300 p ON p.B1_COD = s.D3_COD
        WHERE s.D_E_L_E_T_ <> '*'
          AND p.D_E_L_E_T_ <> '*'
          AND s.D3_TM = '001'
          AND NVL(s.D3_ESTORNO,' ') <> 'S'
          AND s.D3_COD LIKE 'PA02%'
        ORDER BY s.D3_EMISSAO, s.D3_COD, s.D3_DOC, s.D3_TM
    """,
        conn,
    )
len(ingresos_azucar), ingresos_azucar.head(3)

# --------- INGRESO_ALCOHOL (PA01%) ----------
with engine.begin() as conn:
    ingreso_alcohol = pd.read_sql(
        """
        SELECT s.D3_COD                              AS "CODIGO",
               p.B1_DESC                             AS "DESCRIPCION",
               s.D3_UM                               AS "UM",
               s.D3_QUANT                            AS "CANTIDAD",
               s.D3_QTSEGUM                          AS "CANTIDAD L",
               s.D3_DOC                              AS "DOCUMENTO",
               s.D3_EMISSAO                          AS "EMISION",
               s.D3_CUSTO1                           AS "COSTO_BS",
               s.D3_USUARIO                          AS "USUARIO_INGRESO"
        FROM SD3300 s
        JOIN SB1300 p ON p.B1_COD = s.D3_COD
        WHERE s.D_E_L_E_T_ <> '*'
          AND p.D_E_L_E_T_ <> '*'
          AND s.D3_TM = '001'
          AND NVL(s.D3_ESTORNO,' ') <> 'S'
          AND s.D3_COD LIKE 'PA01%'
        ORDER BY s.D3_EMISSAO, s.D3_COD, s.D3_DOC, s.D3_TM
    """,
        conn,
    )
len(ingreso_alcohol), ingreso_alcohol.head(3)

(24,
      codigo        descripcion  um  cantidad  CANTIDAD L documento    emision  \
 0  PA010001  ALCOHOL HIDRATADO  LT   85173.0    85173.48      None 2020-03-01   
 1  PA010001  ALCOHOL HIDRATADO  LT   58110.0    58110.90      None 2020-03-02   
 2  PA010001  ALCOHOL HIDRATADO  LT   57423.0    57423.12      None 2020-03-03   
 
    costo_bs usuario_ingreso  
 0       NaN            None  
 1       NaN            None  
 2       NaN            None  )

Resumen de reglas de negocio aplicadas

- Ingreso de producción: D3_TM = '001'.

- No revertido: D3_ESTORNO <> 'S'.

- Borrado lógico: D_E_L_E_T_ = ' '.

- Catálogo: D3_COD debe existir en SB1300 (join).

- Coherencia UM:

    - Alcohol PA01% → LT (y se refleja como “CANTIDAD L”).

    - Azúcar PA02% → QQ o BL (y “CANTIDAD QQ” calcula según UM).

- Costeo: D3_CUSTO1 generado acorde a UM (no unitario).

- Fechas: se generan en un rango y se ordenan idéntico a tus SQL.